# Context-Level Document Correction
## Serial Python & 3 Apache SPARK Implementations
***

In [ ]:
######################
#
# Submission by Gioia Dominedo (Harvard ID: 40966234) for
# CS 205 - Computing Foundations for Computational Science
# 
# This is part of a joint project with Kendrick Lo that includes a
# separate component for word-level checking. This script includes 
# serial Python code for context-level spell-checking adapted
# from third party algorithms (Symspell and Viterbi algorithms),
# as well as three different SPARK implementations of the same code.
#
# The following were also used as references:
# Peter Norvig, How to Write a Spelling Corrector
#   (http://norvig.com/spell-correct.html)
# Peter Norvig, Natural Language Corpus Data: Beautiful Data
#   (http://norvig.com/ngrams/ch14.pdf)
#
######################

In [ ]:
######################
#
# SUMMARY OF CONTEXT-LEVEL CORRECTION LOGIC - VITERBI ALGORITHM
#
# v 1.0 last revised 6 Dec 2015
#
# Each sentence is modeled as a hidden Markov model. Prior
# probabilities (for first words in the sentences) and transition
# probabilities (for all subsequent words) are calculated when
# generating the main dictionary, using the same corpus. Emission
# probabilities are generated on the fly by parameterizing a Poisson 
# distribution with the edit distance between words and suggested
# corrections.
#
# The state space of possible corrections for each word is generated
# using logic based on the Symspell spell-checker (see below for more
# detail on Symspell). Valid suggestions must: (a) be 'real' words;
# (b) appear at least 100 times in the corpus used to generate the
# dictionary; (c) be one of the top 10 suggestions, based on frequency
# and edit distance. This simplification ensures that the state space
# remains manageable, though it does reduce the accuracy of the
# suggested corrections.
#
# All probabilities are stored in log-space to avoid underflow. Pre-
# defined minimum values are used for words that are not present in
# the dictionary and/or probability tables.
#
# More detail on the various implementations is included below.
#
######################

In [ ]:
import re
import math
from scipy.stats import poisson
import time
import itertools

In [ ]:
import findspark
import os
findspark.init()
import pyspark
sc = pyspark.SparkContext()
sc.setLogLevel('ERROR')

## Serial Implementation
***

### Pre-processing

In [ ]:
######################
#
# PRE-PROCESSING STEPS
#
# The pre-processing steps have been adapted from the dictionary
# creation of the word-level spellchecker, which in turn was based on
# SymSpell, a Symmetric Delete spelling correction algorithm
# developed by Wolf Garbe and originally written in C#. More detail
# on SymSpell is included in the word-level spellcheck documentation.
#
# The main modifications to the word-level spellchecker pre-
# processing stages are to create the additional outputs that are
# required for the context-level checking, and to eliminate redundant
# outputs that are not necessary.
#
# The outputs of the pre-processing stage are:
#
# - dictionary: A dictionary that combines both words present in the
# corpus and other words that are within a given 'delete distance'. 
# The format of the dictionary is:
# {word: ([list of words within the given 'delete distance'], 
# word count in corpus)}
#
# - start_prob: A dictionary with key, value pairs that correspond to
# (word, probability of the word being the first word in a sentence)
#
# - transition_prob: A dictionary of dictionaries that stores the
# probability of a given word following another. The format of the
# dictionary is:
# {previous word: {word1 : P(word1|prevous word), word2 : 
# P(word2|prevous word), ...}}
#
# - default_start_prob: A benchmark probability of a word being at
# the start of a sentence, set to 1 / # of words at the beginning of
# sentences. This ensures that all previously unseen words at the
# beginning of sentences are not corrected unnecessarily.
#
# - default_transition_prob: A benchmark probability of a word being
# seen, given the previous word in the sentence, also set to 1 / # of
# transitions in corpus. This ensures that all previously unseen
# transitions are not corrected unnecessarily.
#
######################

def get_deletes_list(w, max_edit_distance):
    '''
    Given a word, derive strings with up to max_edit_distance
    characters deleted.
    '''
    deletes = []
    queue = [w]
    for d in range(max_edit_distance):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes

def create_dictionary_entry(w, dictionary, max_edit_distance):
    '''
    Add a word and its derived deletions to the dictionary.
    Dictionary entries are of the form:
    ([list of suggested corrections], frequency of word in corpus)
    '''

    new_real_word_added = False
    
    # check if word is already in dictionary
    if w in dictionary:
        # increment count of word in corpus
        dictionary[w] = (dictionary[w][0], dictionary[w][1] + 1)
    else:
        # create new entry in dictionary
        dictionary[w] = ([], 1)  
        
    if dictionary[w][1]==1:
        
        # first appearance of a word in the corpus
        # note: word may already be in dictionary as a derived word
        # (e.g. by deleting character from a real word) but the
        # word counter frequency is not incremented in those cases
        
        new_real_word_added = True
        deletes = get_deletes_list(w, max_edit_distance)
        
        for item in deletes:
            if item in dictionary:
                # add (correct) word to delete's suggested correction
                # list if not already there
                if item not in dictionary[item][0]:
                    dictionary[item][0].append(w)
            else:
                # note: frequency of word in corpus is not incremented
                dictionary[item] = ([w], 0)  
        
    return new_real_word_added

def pre_processing(fname, max_edit_distance=3):
    '''
    Load a text file and use it to create a dictionary and
    to calculate start probabilities and transition probabilities. 
    '''

    dictionary = dict()
    start_prob = dict()
    transition_prob = dict()
    word_count = 0
    transitions = 0
    
    with open(fname) as file:    
        
        for line in file:
            
            # process each sentence separately
            for sentence in line.replace('?','.').replace('!','.').split('.'):
                
                # separate by words by non-alphabetical characters
                words = re.findall('[a-z]+', sentence.lower())      
                
                for w, word in enumerate(words):
                    
                    # create/update dictionary entry
                    if create_dictionary_entry(
                        word, dictionary, max_edit_distance):
                            word_count += 1
                        
                    # update probabilities for Hidden Markov Model
                    if w == 0:

                        # probability of a word being at the
                        # beginning of a sentence
                        if word in start_prob:
                            start_prob[word] += 1
                        else:
                            start_prob[word] = 1
                    else:
                        
                        # probability of transitionining from one
                        # word to another
                        # dictionary format:
                        # {previous word: {word1 : P(word1|prevous
                        # word), word2 : P(word2|prevous word)}}
                        
                        # check whether prior word is present
                        # - create if not
                        if words[w - 1] not in transition_prob:
                            transition_prob[words[w - 1]] = dict()
                            
                        # check whether current word is present
                        # - create if not
                        if word not in transition_prob[words[w - 1]]:
                            transition_prob[words[w - 1]][word] = 0
                            
                        # update value
                        transition_prob[words[w - 1]][word] += 1
                        transitions += 1
                    
    # convert counts to log-probabilities, to avoid underflow in
    # later calculations (note: natural logarithm, not base-10)

    # also calculate (smalle) default probabilities for words that 
    # have not already been seen
    
    # probability of a word being at the beginning of a sentence
    total_start_words = float(sum(start_prob.values()))
    default_start_prob = math.log(1/total_start_words)
    start_prob.update( 
        {k: math.log(v/total_start_words)
         for k, v in start_prob.items()})
    
    # probability of transitioning from one word to another
    default_transition_prob = math.log(1./transitions)
    transition_prob.update(
        {k: {k1: math.log(float(v1)/sum(v.values()))
             for k1, v1 in v.items()} 
         for k, v in transition_prob.items()})

    # output summary statistics
    print 'Total unique words in corpus: %i' % word_count
    print 'Total items in dictionary: %i' \
        % len(dictionary)
    print '  Edit distance for deletions: %i' % max_edit_distance
    print 'Total unique words at the start of a sentence: %i' \
        % len(start_prob)
    print 'Total unique word transitions: %i' % len(transition_prob)
        
    return dictionary, start_prob, default_start_prob, \
        transition_prob, default_transition_prob

### Document correction

In [ ]:
######################
#
# SPELL-CHECKING - VITERBI ALGORITHM
#
# The below functions are used to read in a text file, break it down
# into individual sentences, and then carry out context-based spell-
# checking on each sentence in turn. In cases where the 'suggested'
# word does not match the actual word in the text, both the original
# and the suggested sentences are printed/outputed to file.
#
# Probabilistic model:
#
# Each sentence is modeled as a hidden Markov model, where the
# hidden states are the words that the user intended to type, and
# the emissions are the words that were actually typed.
#
# For each word in a sentence, we can define:
#
# - emission probabilities: P(observed word|intended word)
#
# - prior probabilities (for first words in sentences only):
# P(being the first word in a sentence)
#
# - transition probabilities (for all subsequent words):
# P(intended word|previous intended word)
#
# Prior and transition probabilities were calculated in the pre-
# processing steps above, using the same corpus as the dictionary.
# 
# Emission probabilities are calculated on the fly using a Poisson
# distribution as follows:
# P(observed word|intended word) = PMF of Poisson(k, l), where
# k = edit distance between word typed and word intended, and l=0.01.
# Both the overall approach and the parameter of l=0.01 are based on
# the 2015 lecture notes from AM207 Stochastic Optimization.
# Various parameters for lambda between 0 and 1 were tested, which
# confirmed that 0.01 yields the most accurate word suggestions.
#
# All probabilities are stored in log-space to avoid underflow. Pre-
# defined minimum values (also defined at the pre-processing stage)
# are used for words that are not present in the dictionary and/or
# probability tables.
#
# Algorithm:
#
# The spell-checking itself is carried out using a modified version
# of the Viterbi algorithm, which yields the most likely sequence of
# hidden states, i.e. the most likely sequence of words that form a
# sentence. The main difference to the 'standard' Viterbi algorithm
# is that the state space (i.e. the list of possible corrections) is
# generated (and therefore varies) for each word. This is in contrast
# to the alternative of considering the state space of all possible
# words in the dictionary for every word that is checked, which would
# be intractable for larger dictionaries.
#
# Example:
#
# The algorithm is best illustrated by way of an example.
#
# Suppose that we are checking the sentence 'This is ax test.'
# The emissions for the entire sentence are 'This is ax test.' and
# the hidden states for the entire sentence are 'This is a test.'
#
# As a pre-processing step, we convert everything to lowercase,
# eliminate punctuation, and break the sentence up into a list of
# words: ['this', 'is', 'ax', 'text']
# This list is passed as a parameter to the viterbi function.
#
# The algorithm tackles each word in turn, starting with 'this'.
#
# We first use get_suggestions to obtain a list of all words that
# may have been intended instead of 'this', i.e. all possible hidden
# states (intended words) for the emission (word typed).
#
# get_suggestions returns the 10 most likely corrections:
# - 1 word with an edit distance of 0
#   ['this']
# - 3 words with an edit distance of 1
#   ['his', 'thus', 'thin']
# - 6 words with an edit distance of 2 
#   ['the', 'that', 'is', 'him', 'they', 'their']
# 
# These 10 words represent our state space, i.e. possible words that
# may have been intended, and are referred to below as the list of
# possible corrections. They each have an emission probability equal
# to the PMF of Poisson(edit distance, 0.01).
#
# For each word in the list of possible corrections, we calculate:
# P(word starting a sentence) * P(observed 'this'|intended word)
# This is a simple application of Bayes' rule: by normalizing the
# probabilities we obtain P(intended word|oberved 'this') for
# each of the 10 words.
#
# We store the word-probability pairs for future use, and move on to
# the next word. 
#
# After the first word, all subsequent words are treated as follows.
#
# The second word in our test sentence is 'is'. Once again, we use
# get_suggestions to obtain a list of all words that may have been
# intended. get_suggestions returns the 10 most likely suggestions:
# - 1 word with an edit distance of 0
#   ['is']
# - 9 words with an edit distance of 1
#   ['in', 'it', 'his', 'as', 'i', 's', 'if', 'its', 'us']
# These 10 words represent our state space for the second word.
#
# For each word in the current list of possible corrections, we loop
# through all the words in the previous list of possible corrections,
# and calculate:
#    probability(previous suggested word) 
#    * P(current suggested word|previous suggested word)
#    * P(typing 'is'|meaning to type current suggested word)
# We determine which previous word maximizes this calculation and
# store that 'path' and probability for each current suggested word.
#
# For example, suppose that we are considering the possibility that
# 'is' was indeed intended to be 'is'. We then calculate: 
#    probability(previous suggested word)
#    * P('is'|previous suggested word) * P('is'|'is')
# for all previous suggested words, and discover that the previous
# suggested word 'this' maximizes the above calculation. We therefore
# store 'this is' as the optimal path for the suggested correction
# 'is' and the above (normalized) probability associated with this
# path.
#
# If the sentence had been only 2 words long, then at this point we
# would return the path that maximizes the most probability for the
# most recent step (word).
#
# As it is not, we repeat the previous steps for 'ax' and 'test',
# and then return the path that is associated with the highest
# probability at the last step.
#
######################

def dameraulevenshtein(seq1, seq2):
    '''
    Calculate the Damerau-Levenshtein distance between sequences.

    codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1
    matrix. However, only the current and two previous rows are
    needed at once, so we only store those.

    Same code as word-level checking.
    '''
    
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    
    for x in xrange(len(seq1)):
        
        twoago, oneago, thisrow = \
            oneago, thisrow, [0] * len(seq2) + [x + 1]
        
        for y in xrange(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)

            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
                
    return thisrow[len(seq2) - 1]

def get_suggestions(string, dictionary, max_edit_distance, 
                    longest_word_length=20, min_count=100, max_sug=10):
    '''
    Return list of suggested corrections for potentially incorrectly
    spelled word.

    Code based on get_suggestions function from word-level checking,
    with the addition of the min_count and max_sug parameters.
    - min_count: minimum number of times a word must have appeared
    in the dictionary corpus to be considered a valid suggestion
    - max_sug: number of suggestions that are returned (ranked by
    frequency of appearance in dictionary corpus and edit distance
    from word being checked)

    These changes were imposed in order to ensure that the problem
    remains tractable when checking very large documents. In practice,
    the "correct" suggestion is almost always amongst the top ten.

    '''
    
    if (len(string) - longest_word_length) > max_edit_distance:
        # to ensure Viterbi can keep running -- use the word itself
        return [(string, 0)]
    
    suggest_dict = {}
    
    queue = [string]
    q_dictionary = {}  # items other than string that we've checked
    
    while len(queue)>0:
        q_item = queue[0]  # pop
        queue = queue[1:]
        
        # process queue item
        if (q_item in dictionary) and (q_item not in suggest_dict):
            if (dictionary[q_item][1]>0):
            # word is in dictionary, and is a word from the corpus,
            # and not already in suggestion list so add to suggestion
            # dictionary, indexed by the word with value (frequency
            # in corpus, edit distance)
            # note: q_items that are not the input string are shorter
            # than input string since only deletes are added (unless
            # manual dictionary corrections are added)
                assert len(string)>=len(q_item)
                suggest_dict[q_item] = \
                    (dictionary[q_item][1], len(string) - len(q_item))
            
            # the suggested corrections for q_item as stored in
            # dictionary (whether or not q_item itself is a valid
            # word or merely a delete) can be valid corrections
            for sc_item in dictionary[q_item][0]:
                if (sc_item not in suggest_dict):
                    
                    # compute edit distance
                    # suggested items should always be longer (unless
                    # manual corrections are added)
                    assert len(sc_item)>len(q_item)
                    # q_items that are not input should be shorter
                    # than original string 
                    # (unless manual corrections added)
                    assert len(q_item)<=len(string)
                    if len(q_item)==len(string):
                        assert q_item==string
                        item_dist = len(sc_item) - len(q_item)

                    # item in suggestions list should not be the same
                    # as the string itself
                    assert sc_item!=string           
                    # calculate edit distance using Damerau-
                    # Levenshtein distance
                    item_dist = dameraulevenshtein(sc_item, string)
                    
                    if item_dist<=max_edit_distance:
                        # should already be in dictionary if in
                        # suggestion list
                        assert sc_item in dictionary  
                        # trim list to contain state space
                        if (dictionary[q_item][1]>0): 
                            suggest_dict[sc_item] = \
                                (dictionary[sc_item][1], item_dist)
        
        # now generate deletes (e.g. a substring of string or of a
        # delete) from the queue item as additional items to check
        # -- add to end of queue
        assert len(string)>=len(q_item)
        if (len(string)-len(q_item))<max_edit_distance \
            and len(q_item)>1:
            for c in range(len(q_item)): # character index        
                word_minus_c = q_item[:c] + q_item[c+1:]
                if word_minus_c not in q_dictionary:
                    queue.append(word_minus_c)
                    # arbitrary value to identify we checked this
                    q_dictionary[word_minus_c] = None

    # return list of suggestions: (correction, edit distance)
    
    # only include words that have appeared a minimum number of times
    # note: make sure that we do not lose the original word
    as_list = [i for i in suggest_dict.items() 
               if (i[1][0]>min_count or i[0]==string)]
    
    # only include the most likely suggestions (based on frequency
    # and edit distance from original word)
    trunc_as_list = sorted(as_list, 
            key = lambda (term, (freq, dist)): (dist, -freq))[:max_sug]
    
    if len(trunc_as_list)==0:
        # to ensure Viterbi can keep running
        # -- use the word itself if no corrections are found
        return [(string, 0)]
        
    else:
        # drop the word frequency - not needed beyond this point
        return [(i[0], i[1][1]) for i in trunc_as_list]

    '''
    Output format:
    get_suggestions('file', dictionary)
    [('file', 0), ('five', 1), ('fire', 1), ('fine', 1), ('will', 2),
    ('time', 2), ('face', 2), ('like', 2), ('life', 2), ('while', 2)]
    '''
    
def get_emission_prob(edit_dist, poisson_lambda=0.01):
    '''
    The emission probability, i.e. P(observed word|intended word)
    is approximated by a Poisson(k, l) distribution, where 
    k=edit distance between the observed word and the intended
    word and l=0.01.
    
    Both the overall approach and the parameter of l=0.01 are based on
    the 2015 lecture notes from AM207 Stochastic Optimization.
    Various parameters for lambda between 0 and 1 were tested, which
    confirmed that 0.01 yields the most accurate word suggestions.
    '''
    
    return math.log(poisson.pmf(edit_dist, poisson_lambda))

######################
#
# Multiple helper functions are used to avoid KeyErrors when
# attempting to access values that are not present in dictionaries,
# in which case the previously specified default value is returned.
#
######################

def get_start_prob(word, start_prob, default_start_prob):
    '''
    P(word being at the beginning of a sentence)
    '''
    try:
        return start_prob[word]
    except KeyError:
        return default_start_prob
    
def get_transition_prob(cur_word, prev_word, 
                        transition_prob, default_transition_prob):
    '''
    P(word|previous word)
    '''
    try:
        return transition_prob[prev_word][cur_word]
    except KeyError:
        return default_transition_prob

def get_path_prob(prev_word, prev_path_prob):
    '''
    P(previous path)
    '''
    try:
        return prev_path_prob[prev_word]
    except KeyError:
        return math.log(math.exp(min(prev_path_prob.values()))/2.)  
    
def viterbi(words, dictionary, start_prob, default_start_prob, 
            transition_prob, default_transition_prob, max_edit_distance):
    '''
    Determine the most likely (intended) sequence, based on the
    observed sequence. Full details in preamble above.
    '''

    V = [{}]
    path = {}
    path_context = []
    
    # character level correction - used to determine state space
    corrections = get_suggestions(words[0], dictionary, max_edit_distance)
        
    # Initialize base cases (first word in the sentence)
    for sug_word in corrections:
        
        # compute the value for all possible starting states
        V[0][sug_word[0]] = math.exp(
            get_start_prob(sug_word[0], start_prob, 
                           default_start_prob)
            + get_emission_prob(sug_word[1]))
        
        # remember all the different paths (only one word so far)
        path[sug_word[0]] = [sug_word[0]]
 
    # normalize for numerical stability
    path_temp_sum = sum(V[0].values())
    V[0].update({k: math.log(v/path_temp_sum) 
                 for k, v in V[0].items()})
    
    # keep track of previous state space
    prev_corrections = [i[0] for i in corrections]
    
    # return if the sentence only has one word
    if len(words) == 1:
        path_context = [max(V[0], key=lambda i: V[0][i])]
        return path_context

    # run Viterbi for all subsequent words in the sentence
    for t in range(1, len(words)):

        V.append({})
        new_path = {}
        
        # character level correction
        corrections = get_suggestions(words[t], dictionary, max_edit_distance)
 
        for sug_word in corrections:
        
            sug_word_emission_prob = get_emission_prob(sug_word[1])
            
            # compute the probabilities associated with all previous
            # states (paths), only keep the maximum
            (prob, word) = max(
                (get_path_prob(prev_word, V[t-1]) 
                + get_transition_prob(sug_word[0], prev_word, 
                    transition_prob, default_transition_prob)
                + sug_word_emission_prob, prev_word) 
                               for prev_word in prev_corrections)

            # save the maximum probability for each state
            V[t][sug_word[0]] = math.exp(prob)
            
            # store the full path that results in this probability
            new_path[sug_word[0]] = path[word] + [sug_word[0]]
        
        # normalize for numerical stability
        path_temp_sum = sum(V[t].values())
        V[t].update({k: math.log(v/path_temp_sum) 
                     for k, v in V[t].items()})
        
        # keep track of previous state space
        prev_corrections = [i[0] for i in corrections]
 
        # don't need to remember the old paths
        path = new_path
     
    # after all iterations are completed, look up the word with the
    # highest probability
    (prob, word) = max((V[t][sug_word[0]], sug_word[0]) 
                       for sug_word in corrections)

    # look up the full path associated with this word
    path_context = path[word]

    return path_context

def correct_document_context(fname, dictionary, 
                             start_prob, default_start_prob,
                             transition_prob, default_transition_prob,
                             max_edit_distance=3, display_results=False):
    
    '''
    Load a text file and spell-check each sentence using the
    dictionary and probability tables that were created in the
    pre-processing stage.

    Suggested corrections are either printed to the screen or
    saved in a log file, depending on the settings.
    '''

    doc_word_count = 0
    corrected_word_count = 0
    sentence_errors_list = []
    total_sentences = 0
    
    with open(fname) as file:
        
        for i, line in enumerate(file):
            
            for sentence in line.replace('?','.').replace('!','.').split('.'):
                
                # separate by words by non-alphabetical characters
                words = re.findall('[a-z]+', sentence.lower())  
                doc_word_count += len(words)
                
                if len(words) > 0:
                
                    # run Viterbi algorithm for each sentence and
                    # obtain most likely correction (may be the same
                    # as the original sentence)
                    suggestion = viterbi(words, dictionary,
                                start_prob, default_start_prob, 
                                transition_prob, default_transition_prob,
                                max_edit_distance)

                    # display sentences with suggested changes
                    if words != suggestion:
                        
                        # keep track of all potential errors
                        sentence_errors_list.append([total_sentences, 
                            (words, suggestion)])

                        # update count of corrected words
                        corrected_word_count += \
                        sum([words[j]!=suggestion[j] 
                             for j in range(len(words))])
                        
                    # used for display purposes
                    total_sentences += 1
  
    # print suggested corrections
    if display_results:
        for sentence in sentence_errors_list:
            print 'Sentence %i: %s --> %s' % (sentence[0],
                ' '.join(sentence[1][0]), ' '.join(sentence[1][1]))
            print '-----'
    
    # output suggested corrections to file
    else:
        f = open('spell-log.txt', 'w')
        for sentence in sentence_errors_list:
            f.write('Sentence %i: %s --> %s\n' % (sentence[0], 
                ' '.join(sentence[1][0]), ' '.join(sentence[1][1])))
        f.close()
            
    # display summary statistics
    print 'Total words checked: %i' % doc_word_count
    print 'Total potential errors found: %i' % corrected_word_count

### Sample performance
The performance of the serial algorithm provides a benchmark against which to compare the subsequent SPARK implementations.

In [ ]:
dictionary_file = 'testdata/big.txt'
check_file = 'testdata/yelp100reviews.txt'

In [ ]:
print 'Pre-processing with %s...' % dictionary_file

start_time = time.time()

dictionary, start_prob, default_start_prob, \
transition_prob, default_transition_prob \
= pre_processing(dictionary_file)

run_time = time.time() - start_time

print '-----'
print '%.2f seconds to run' % run_time
print '-----'

```
Pre-processing with testdata/big.txt...
Total unique words in corpus: 29157
Total items in dictionary: 2151998
  Edit distance for deletions: 3
Total unique words at the start of a sentence: 15356
Total unique word transitions: 27086
-----
32.47 seconds to run
-----
```

In [ ]:
print 'Spell-checking %s...' % check_file

start_time = time.time()

correct_document_context(check_file, dictionary,
                         start_prob, default_start_prob, 
                         transition_prob, default_transition_prob)

run_time = time.time() - start_time

print '-----'
print '%.2f seconds to run' % run_time
print '-----'

```
Spell-checking testdata/yelp100reviews.txt...
Total words checked: 12029
Total potential errors found: 1735
-----
303.63 seconds to run
-----
```
Sample output with suggested corrections <a href='sample/context-spell-log-yelp.txt'>here</a>.

<div class="alert alert-info">
<p><strong>NOTE:</strong> Three different SPARK implementations were developed and tested. Due to variations in approaches, some of the helper functions vary slightly across implementations but may have the same name. To avoid potential errors, please run the relevant code blocks in each sections before attempting to run the sample code.</p>
<p>Please note that runtimes tend to be slightly longer in the iPython notebook when compared to the Python scripts. All runtime results below were derived by running the Python scripts.</p>
</div>

## SPARK Implementation # 1 - Naive Parallelization
***
For this implementation we create an RDD where each element corresponds to a sentence from the document to be checked, and then use a map operation to call the Viterbi function for each sentence.

This approach takes advantage of parallelization by splitting the sentences among the workers (i.e. increasing the number of workers will improve the runtime), but does not parallelize the algorithm itself.

Note: all the functions related to the Viterbi algorithm are the same as those used in the serial implementation.

<img src='img/contextSPARK1.jpg' width=50% height=50%>

### Pre-processing

In [ ]:
######################
#
# PRE-PROCESSING STEPS
#
# The pre-processing steps have been adapted from the dictionary
# creation of the word-level spellchecker, which in turn was based on
# SymSpell, a Symmetric Delete spelling correction algorithm
# developed by Wolf Garbe and originally written in C#. More detail
# on SymSpell is included in the word-level spellcheck documentation.
#
# The main modifications to the word-level spellchecker pre-
# processing stages are to create the additional outputs that are
# required for the context-level checking, and to eliminate redundant
# outputs that are not necessary.
#
# The outputs of the pre-processing stage are:
#
# - dictionary: A dictionary that combines both words present in the
# corpus and other words that are within a given 'delete distance'. 
# The format of the dictionary is:
# {word: ([list of words within the given 'delete distance'], 
# word count in corpus)}
#
# - start_prob: A dictionary with key, value pairs that correspond to
# (word, probability of the word being the first word in a sentence)
#
# - transition_prob: A dictionary of dictionaries that stores the
# probability of a given word following another. The format of the
# dictionary is:
# {previous word: {word1 : P(word1|prevous word), word2 : 
# P(word2|prevous word), ...}}
#
# - default_start_prob: A benchmark probability of a word being at
# the start of a sentence, set to 1 / # of words at the beginning of
# sentences. This ensures that all previously unseen words at the
# beginning of sentences are not corrected unnecessarily.
#
# - default_transition_prob: A benchmark probability of a word being
# seen, given the previous word in the sentence, also set to 1 / # of
# transitions in corpus. This ensures that all previously unseen
# transitions are not corrected unnecessarily.
#
######################

def get_deletes_list(w, max_edit_distance):
    '''
    Given a word, derive strings with up to max_edit_distance
    characters deleted. 

    The list is generally of the same magnitude as the number of
    characters in a word, so it does not make sense to parallelize
    this function. Instead, we use Python to create the list.
    '''
    deletes = []
    queue = [w]
    for d in range(max_edit_distance):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes

def get_transitions(sentence):
    '''
    Helper function: converts a sentence into all two-word pairs.
    Output format is a list of tuples.
    e.g. 'This is a test' >> ('this', 'is'), ('is', 'a'), ('a', 'test')
    ''' 
    if len(sentence)<2:
        return None
    else:
        return [((sentence[i], sentence[i+1]), 1) 
                for i in range(len(sentence)-1)]
    
def map_transition_prob(vals):
    '''
    Helper function: calculates conditional probabilities for all word
    pairs, i.e. P(word|previous word)
    '''
    total = float(sum(vals.values()))
    return {k: math.log(v/total) for k, v in vals.items()}

def parallel_create_dictionary(fname, max_edit_distance=3, 
                                num_partitions=6):
    '''
    Load a text file and use it to create a dictionary and
    to calculate start probabilities and transition probabilities. 
    '''
    
    # Note: this function makes use of multiple accumulators to keep
    # track of the words that are being processed. An alternative 
    # implementation that wraps accumulators in helper functions was
    # also tested, but did not yield any noticeable improvements.

    ############
    #
    # load file & initial processing
    #
    ############
    
    # http://stackoverflow.com/questions/22520932/python-remove-all-non-alphabet-chars-from-string
    regex = re.compile('[^a-z ]')

    # load file contents and convert into one long sequence of words
    # RDD format: 'line 1', 'line 2', 'line 3', ...
    # cache because this RDD is used in multiple operations 
    make_all_lower = sc.textFile(fname) \
            .map(lambda line: line.lower()) \
            .filter(lambda x: x!='').cache()
    
    # split into individual sentences and remove other punctuation
    # RDD format: [words of sentence 1], [words of sentence 2], ...
    # cache because this RDD is used in multiple operations 
    split_sentence = make_all_lower.flatMap(lambda 
        line: line.replace('?','.').replace('!','.').split('.')) \
             .map(lambda sentence: regex.sub(' ', sentence)) \
             .map(lambda sentence: sentence.split()) \
             .filter(lambda x: x!=[]).cache()
    
    ############
    #
    # generate start probabilities
    #
    ############
    
    # extract all words that are at the beginning of sentences
    # RDD format: 'word1', 'word2', 'word3', ...
    start_words = split_sentence.map(lambda sentence: sentence[0] 
        if len(sentence)>0 else None) \
            .filter(lambda word: word!=None)
    
    # add a count to each word
    # RDD format: ('word1', 1), ('word2', 1), ('word3', 1), ...
    # note: partition here because we are using words as keys for
    # the first time - yields a small but consistent improvement in
    # runtime (~2-3 sec for big.txt)
    # cache because this RDD is used in multiple operations
    count_start_words_once = start_words.map(lambda word: (word, 1)) \
            .partitionBy(num_partitions).cache()

    # use accumulator to count the number of start words processed
    accum_total_start_words = sc.accumulator(0)
    count_start_words_once.foreach(lambda x: accum_total_start_words.add(1))
    total_start_words = float(accum_total_start_words.value)
    
    # reduce into count of unique words at the start of sentences
    # RDD format: ('word1', frequency), ('word2', frequency), ...
    unique_start_words = count_start_words_once.reduceByKey(lambda a, b: a + b)
    
    # convert counts to log-probabilities
    # RDD format: ('word1', log-prob of word1), 
    #             ('word2', log-prob of word2), ...
    start_prob_calc = unique_start_words.mapValues(lambda v: 
        math.log(v/total_start_words))
    
    # get default start probabilities (for words not in corpus)
    default_start_prob = math.log(1/total_start_words)
    
    # store start probabilities as a dictionary (i.e. a lookup table)
    # note: given the spell-checking algorithm, this cannot be maintained
    # as an RDD as it is not possible to map within a map
    start_prob = start_prob_calc.collectAsMap()
    
    ############
    #
    # generate transition probabilities
    #
    ############
    
    # note: various partitioning strategies were attempted for this
    # portion of the function, but they failed to yield significant
    # improvements in performance.

    # focus on continuous word pairs within the sentence
    # e.g. "this is a test" -> "this is", "is a", "a test"
    # note: as the relevant probability is P(word|previous word)
    # the tuples are ordered as (previous word, word)

    # extract all word pairs within a sentence and add a count
    # RDD format: (('word1', 'word2'), 1), (('word2', 'word3'), 1), ...
    # cache because this RDD is used in multiple operations 
    other_words = split_sentence.map(lambda sentence: 
        get_transitions(sentence)) \
            .filter(lambda x: x!=None) \
            .flatMap(lambda x: x).cache()

    # use accumulator to count the number of transitions (word pairs)
    accum_total_other_words = sc.accumulator(0)
    other_words.foreach(lambda x: accum_total_other_words.add(1))
    total_other_words = float(accum_total_other_words.value)
    
    # reduce into count of unique word pairs
    # RDD format: (('word1', 'word2'), frequency), 
    #             (('word2', 'word3'), frequency), ...
    unique_other_words = other_words.reduceByKey(lambda a, b: a + b)
    
    # aggregate by (and change key to) previous word
    # RDD format: ('previous word', {'word1': word pair count, 
    #                                'word2': word pair count}}), ...
    other_words_collapsed = unique_other_words.map(lambda x: 
        (x[0][0], (x[0][1], x[1]))) \
            .groupByKey().mapValues(dict)

    # note: the above line of code is the slowest in the function
    # (8.6 MB shuffle read and 4.5 MB shuffle write for big.txt)
    # An alternative approach that aggregates lists with reduceByKey was
    # attempted, but did not yield noticeable improvements in runtime.
    
    # convert counts to log-probabilities
    # RDD format: ('previous word', {'word1': log-prob of pair, 
    #                                 word2: log-prob of pair}}), ...
    transition_prob_calc = other_words_collapsed.mapValues(lambda v: 
        map_transition_prob(v))

    # get default transition probabilities (for word pairs not in corpus)
    default_transition_prob = math.log(1/total_other_words)
    
    # store transition probabilities as a dictionary (i.e. a lookup table)
    # note: given the spell-checking algorithm, this cannot be maintained
    # as an RDD as it is not possible to map within a map
    transition_prob = transition_prob_calc.collectAsMap()
    
    ############
    #
    # generate dictionary
    #
    ############

    # note: this approach is slightly different from the original SymSpell
    # algorithm, but is more appropriate for a SPARK implementation
    
    # split into individual words (all)
    # RDD format: 'word1', 'word2', 'word3', ...
    # cache because this RDD is used in multiple operations 
    all_words = make_all_lower.map(lambda line: regex.sub(' ', line)) \
            .flatMap(lambda line: line.split()).cache()

    # use accumulator to count the number of words processed
    accum_words_processed = sc.accumulator(0)
    all_words.foreach(lambda x: accum_words_processed.add(1))

    # add a count to each word
    # RDD format: ('word1', 1), ('word2', 1), ('word3', 1), ...
    count_once = all_words.map(lambda word: (word, 1))

    # reduce into counts of unique words - this is the core corpus dictionary
    # (i.e. only words appearing in the file, without 'deletes'))
    # RDD format: ('word1', frequency), ('word2', frequency), ...
    # cache because this RDD is used in multiple operations 
    # note: imposing partitioning at this step yields a small 
    # improvement in runtime (~1 sec for big.txt) by equally
    # balancing elements among workers for subsequent operations
    unique_words_with_count = count_once.reduceByKey(lambda a, b: a + b, 
        numPartitions = num_partitions).cache()
    
    # use accumulator to count the number of unique words
    accum_unique_words = sc.accumulator(0)
    unique_words_with_count.foreach(lambda x: accum_unique_words.add(1))

    # generate list of "deletes" for each word in the corpus
    # RDD format: (word1, [deletes for word1]), (word2, [deletes for word2]), ...
    generate_deletes = unique_words_with_count.map(lambda (parent, count): 
        (parent, get_deletes_list(parent, max_edit_distance)))
    
    # split into all key-value pairs
    # RDD format: (word1, delete1), (word1, delete2), ...
    expand_deletes = generate_deletes.flatMapValues(lambda x: x)
    
    # swap word order and add a zero count (because "deletes" were not
    # present in the dictionary)
    swap = expand_deletes.map(lambda (orig, delete): (delete, ([orig], 0)))
    
    # create a placeholder for each real word
    # RDD format: ('word1', ([], frequency)), ('word2', ([], frequency)), ...
    corpus = unique_words_with_count.mapValues(lambda count: ([], count))

    # combine main dictionary and "deletes" (and eliminate duplicates)
    # RDD format: ('word1', ([deletes for word1], frequency)), 
    #             ('word2', ([deletes for word2], frequency)), ...
    combine = swap.union(corpus)
    
    # store dictionary items and deletes as a dictionary (i.e. a lookup table)
    # note: given the spell-checking algorithm, this cannot be maintained
    # as an RDD as it is not possible to map within a map
    # note: use reduceByKeyLocally to avoid an extra shuffle from reduceByKey
    dictionary = combine.reduceByKeyLocally(lambda a, b: (a[0]+b[0], a[1]+b[1])) 
    
    # output stats
    print 'Total words processed: %i' % accum_words_processed.value
    print 'Total unique words in corpus: %i' % accum_unique_words.value 
    print 'Total items in dictionary (corpus words and deletions): %i' \
        % len(dictionary)
    print '  Edit distance for deletions: %i' % max_edit_distance
    print 'Total unique words at the start of a sentence: %i' \
        % len(start_prob)
    print 'Total unique word transitions: %i' % len(transition_prob)
    
    return dictionary, start_prob, default_start_prob, \
            transition_prob, default_transition_prob

### Document correction

In [ ]:
######################
#
# SPELL-CHECKING - VITERBI ALGORITHM
#
# The below functions are used to read in a text file, break it down
# into individual sentences, and then carry out context-based spell-
# checking on each sentence in turn. In cases where the 'suggested'
# word does not match the actual word in the text, both the original
# and the suggested sentences are printed/outputed to file.
#
# Probabilistic model:
#
# Each sentence is modeled as a hidden Markov model, where the
# hidden states are the words that the user intended to type, and
# the emissions are the words that were actually typed.
#
# For each word in a sentence, we can define:
#
# - emission probabilities: P(observed word|intended word)
#
# - prior probabilities (for first words in sentences only):
# P(being the first word in a sentence)
#
# - transition probabilities (for all subsequent words):
# P(intended word|previous intended word)
#
# Prior and transition probabilities were calculated in the pre-
# processing steps above, using the same corpus as the dictionary.
# 
# Emission probabilities are calculated on the fly using a Poisson
# distribution as follows:
# P(observed word|intended word) = PMF of Poisson(k, l), where
# k = edit distance between word typed and word intended, and l=0.01.
# Both the overall approach and the parameter of l=0.01 are based on
# the 2015 lecture notes from AM207 Stochastic Optimization.
# Various parameters for lambda between 0 and 1 were tested, which
# confirmed that 0.01 yields the most accurate word suggestions.
#
# All probabilities are stored in log-space to avoid underflow. Pre-
# defined minimum values (also defined at the pre-processing stage)
# are used for words that are not present in the dictionary and/or
# probability tables.
#
# Algorithm:
#
# The spell-checking itself is carried out using a modified version
# of the Viterbi algorithm, which yields the most likely sequence of
# hidden states, i.e. the most likely sequence of words that form a
# sentence. The main difference to the 'standard' Viterbi algorithm
# is that the state space (i.e. the list of possible corrections) is
# generated (and therefore varies) for each word. This is in contrast
# to the alternative of considering the state space of all possible
# words in the dictionary for every word that is checked, which would
# be intractable for larger dictionaries.
#
# Example:
#
# The algorithm is best illustrated by way of an example.
#
# Suppose that we are checking the sentence 'This is ax test.'
# The emissions for the entire sentence are 'This is ax test.' and
# the hidden states for the entire sentence are 'This is a test.'
#
# As a pre-processing step, we convert everything to lowercase,
# eliminate punctuation, and break the sentence up into a list of
# words: ['this', 'is', 'ax', 'text']
# This list is passed as a parameter to the viterbi function.
#
# The algorithm tackles each word in turn, starting with 'this'.
#
# We first use get_suggestions to obtain a list of all words that
# may have been intended instead of 'this', i.e. all possible hidden
# states (intended words) for the emission (word typed).
#
# get_suggestions returns the 10 most likely corrections:
# - 1 word with an edit distance of 0
#   ['this']
# - 3 words with an edit distance of 1
#   ['his', 'thus', 'thin']
# - 6 words with an edit distance of 2 
#   ['the', 'that', 'is', 'him', 'they', 'their']
# 
# These 10 words represent our state space, i.e. possible words that
# may have been intended, and are referred to below as the list of
# possible corrections. They each have an emission probability equal
# to the PMF of Poisson(edit distance, 0.01).
#
# For each word in the list of possible corrections, we calculate:
# P(word starting a sentence) * P(observed 'this'|intended word)
# This is a simple application of Bayes' rule: by normalizing the
# probabilities we obtain P(intended word|oberved 'this') for
# each of the 10 words.
#
# We store the word-probability pairs for future use, and move on to
# the next word. 
#
# After the first word, all subsequent words are treated as follows.
#
# The second word in our test sentence is 'is'. Once again, we use
# get_suggestions to obtain a list of all words that may have been
# intended. get_suggestions returns the 10 most likely suggestions:
# - 1 word with an edit distance of 0
#   ['is']
# - 9 words with an edit distance of 1
#   ['in', 'it', 'his', 'as', 'i', 's', 'if', 'its', 'us']
# These 10 words represent our state space for the second word.
#
# For each word in the current list of possible corrections, we loop
# through all the words in the previous list of possible corrections,
# and calculate:
#    probability(previous suggested word) 
#    * P(current suggested word|previous suggested word)
#    * P(typing 'is'|meaning to type current suggested word)
# We determine which previous word maximizes this calculation and
# store that 'path' and probability for each current suggested word.
#
# For example, suppose that we are considering the possibility that
# 'is' was indeed intended to be 'is'. We then calculate: 
#    probability(previous suggested word)
#    * P('is'|previous suggested word) * P('is'|'is')
# for all previous suggested words, and discover that the previous
# suggested word 'this' maximizes the above calculation. We therefore
# store 'this is' as the optimal path for the suggested correction
# 'is' and the above (normalized) probability associated with this
# path.
#
# If the sentence had been only 2 words long, then at this point we
# would return the path that maximizes the most probability for the
# most recent step (word).
#
# As it is not, we repeat the previous steps for 'ax' and 'test',
# and then return the path that is associated with the highest
# probability at the last step.
#
######################

def dameraulevenshtein(seq1, seq2):
    '''
    Calculate the Damerau-Levenshtein distance between sequences.

    codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1
    matrix. However, only the current and two previous rows are
    needed at once, so we only store those.

    Same code as word-level checking.
    '''
    
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    
    for x in xrange(len(seq1)):
        
        twoago, oneago, thisrow = \
            oneago, thisrow, [0] * len(seq2) + [x + 1]
        
        for y in xrange(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)

            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
                
    return thisrow[len(seq2) - 1]

def get_suggestions(string, dictionary, max_edit_distance, 
                    longest_word_length=20, min_count=100, max_sug=10):
    '''
    Return list of suggested corrections for potentially incorrectly
    spelled word.

    Code based on get_suggestions function from word-level checking,
    with the addition of the min_count and max_sug parameters.
    - min_count: minimum number of times a word must have appeared
    in the dictionary corpus to be considered a valid suggestion
    - max_sug: number of suggestions that are returned (ranked by
    frequency of appearance in dictionary corpus and edit distance
    from word being checked)

    These changes were imposed in order to ensure that the problem
    remains tractable when checking very large documents. In practice,
    the "correct" suggestion is almost always amongst the top ten.

    '''
    
    if (len(string) - longest_word_length) > max_edit_distance:
        # to ensure Viterbi can keep running -- use the word itself
        return [(string, 0)]
    
    suggest_dict = {}
    
    queue = [string]
    q_dictionary = {}  # items other than string that we've checked
    
    while len(queue)>0:
        q_item = queue[0]  # pop
        queue = queue[1:]
        
        # process queue item
        if (q_item in dictionary) and (q_item not in suggest_dict):
            if (dictionary[q_item][1]>0):
            # word is in dictionary, and is a word from the corpus,
            # and not already in suggestion list so add to suggestion
            # dictionary, indexed by the word with value (frequency
            # in corpus, edit distance)
            # note: q_items that are not the input string are shorter
            # than input string since only deletes are added (unless
            # manual dictionary corrections are added)
                assert len(string)>=len(q_item)
                suggest_dict[q_item] = \
                    (dictionary[q_item][1], len(string) - len(q_item))
            
            # the suggested corrections for q_item as stored in
            # dictionary (whether or not q_item itself is a valid
            # word or merely a delete) can be valid corrections
            for sc_item in dictionary[q_item][0]:
                if (sc_item not in suggest_dict):
                    
                    # compute edit distance
                    # suggested items should always be longer (unless
                    # manual corrections are added)
                    assert len(sc_item)>len(q_item)
                    # q_items that are not input should be shorter
                    # than original string 
                    # (unless manual corrections added)
                    assert len(q_item)<=len(string)
                    if len(q_item)==len(string):
                        assert q_item==string
                        item_dist = len(sc_item) - len(q_item)

                    # item in suggestions list should not be the same
                    # as the string itself
                    assert sc_item!=string           
                    # calculate edit distance using Damerau-
                    # Levenshtein distance
                    item_dist = dameraulevenshtein(sc_item, string)
                    
                    if item_dist<=max_edit_distance:
                        # should already be in dictionary if in
                        # suggestion list
                        assert sc_item in dictionary  
                        # trim list to contain state space
                        if (dictionary[q_item][1]>0): 
                            suggest_dict[sc_item] = \
                                (dictionary[sc_item][1], item_dist)
        
        # now generate deletes (e.g. a substring of string or of a
        # delete) from the queue item as additional items to check
        # -- add to end of queue
        assert len(string)>=len(q_item)
        if (len(string)-len(q_item))<max_edit_distance \
            and len(q_item)>1:
            for c in range(len(q_item)): # character index        
                word_minus_c = q_item[:c] + q_item[c+1:]
                if word_minus_c not in q_dictionary:
                    queue.append(word_minus_c)
                    # arbitrary value to identify we checked this
                    q_dictionary[word_minus_c] = None

    # return list of suggestions: (correction, edit distance)
    
    # only include words that have appeared a minimum number of times
    # note: make sure that we do not lose the original word
    as_list = [i for i in suggest_dict.items() 
               if (i[1][0]>min_count or i[0]==string)]
    
    # only include the most likely suggestions (based on frequency
    # and edit distance from original word)
    trunc_as_list = sorted(as_list, 
            key = lambda (term, (freq, dist)): (dist, -freq))[:max_sug]
    
    if len(trunc_as_list)==0:
        # to ensure Viterbi can keep running
        # -- use the word itself if no corrections are found
        return [(string, 0)]
        
    else:
        # drop the word frequency - not needed beyond this point
        return [(i[0], i[1][1]) for i in trunc_as_list]

    '''
    Output format:
    get_suggestions('file', dictionary)
    [('file', 0), ('five', 1), ('fire', 1), ('fine', 1), ('will', 2),
    ('time', 2), ('face', 2), ('like', 2), ('life', 2), ('while', 2)]
    '''
    
def get_emission_prob(edit_dist, poisson_lambda=0.01):
    '''
    The emission probability, i.e. P(observed word|intended word)
    is approximated by a Poisson(k, l) distribution, where 
    k=edit distance between the observed word and the intended
    word and l=0.01.
    
    Both the overall approach and the parameter of l=0.01 are based on
    the 2015 lecture notes from AM207 Stochastic Optimization.
    Various parameters for lambda between 0 and 1 were tested, which
    confirmed that 0.01 yields the most accurate word suggestions.
    '''
    
    return math.log(poisson.pmf(edit_dist, poisson_lambda))

######################
#
# Multiple helper functions are used to avoid KeyErrors when
# attempting to access values that are not present in dictionaries,
# in which case the previously specified default value is returned.
#
######################

def get_start_prob(word, start_prob, default_start_prob):
    '''
    P(word being at the beginning of a sentence)
    '''
    try:
        return start_prob[word]
    except KeyError:
        return default_start_prob
    
def get_transition_prob(cur_word, prev_word, 
                        transition_prob, default_transition_prob):
    '''
    P(word|previous word)
    '''
    try:
        return transition_prob[prev_word][cur_word]
    except KeyError:
        return default_transition_prob

def get_path_prob(prev_word, prev_path_prob):
    '''
    P(previous path)
    '''
    try:
        return prev_path_prob[prev_word]
    except KeyError:
        return math.log(math.exp(min(prev_path_prob.values()))/2.)  
    
def viterbi(words, dictionary, start_prob, default_start_prob, 
            transition_prob, default_transition_prob, max_edit_distance):
    '''
    Determine the most likely (intended) sequence, based on the
    observed sequence. Full details in preamble above.
    '''

    V = [{}]
    path = {}
    path_context = []
    
    # character level correction - used to determine state space
    corrections = get_suggestions(words[0], dictionary, max_edit_distance)
        
    # Initialize base cases (first word in the sentence)
    for sug_word in corrections:
        
        # compute the value for all possible starting states
        V[0][sug_word[0]] = math.exp(
            get_start_prob(sug_word[0], start_prob, 
                           default_start_prob)
            + get_emission_prob(sug_word[1]))
        
        # remember all the different paths (only one word so far)
        path[sug_word[0]] = [sug_word[0]]
 
    # normalize for numerical stability
    path_temp_sum = sum(V[0].values())
    V[0].update({k: math.log(v/path_temp_sum) 
                 for k, v in V[0].items()})
    
    # keep track of previous state space
    prev_corrections = [i[0] for i in corrections]
    
    # return if the sentence only has one word
    if len(words) == 1:
        path_context = [max(V[0], key=lambda i: V[0][i])]
        return path_context

    # run Viterbi for all subsequent words in the sentence
    for t in range(1, len(words)):

        V.append({})
        new_path = {}
        
        # character level correction
        corrections = get_suggestions(words[t], dictionary, max_edit_distance)
 
        for sug_word in corrections:
        
            sug_word_emission_prob = get_emission_prob(sug_word[1])
            
            # compute the probabilities associated with all previous
            # states (paths), only keep the maximum
            (prob, word) = max(
                (get_path_prob(prev_word, V[t-1]) 
                + get_transition_prob(sug_word[0], prev_word, 
                    transition_prob, default_transition_prob)
                + sug_word_emission_prob, prev_word) 
                               for prev_word in prev_corrections)

            # save the maximum probability for each state
            V[t][sug_word[0]] = math.exp(prob)
            
            # store the full path that results in this probability
            new_path[sug_word[0]] = path[word] + [sug_word[0]]
        
        # normalize for numerical stability
        path_temp_sum = sum(V[t].values())
        V[t].update({k: math.log(v/path_temp_sum) 
                     for k, v in V[t].items()})
        
        # keep track of previous state space
        prev_corrections = [i[0] for i in corrections]
 
        # don't need to remember the old paths
        path = new_path
     
    # after all iterations are completed, look up the word with the
    # highest probability
    (prob, word) = max((V[t][sug_word[0]], sug_word[0]) 
                       for sug_word in corrections)

    # look up the full path associated with this word
    path_context = path[word]

    return path_context

def get_count_mismatches(sentences):
    '''
    Helper function: compares the original sentence with the sentence
    that has been suggested by the Viterbi algorithm, and calculates
    the number of words that do not match.
    '''
    orig_sentence, sug_sentence = sentences
    count_mismatches = len([(orig_sentence[i], sug_sentence[i]) 
            for i in range(len(orig_sentence))
            if orig_sentence[i]!=sug_sentence[i]])
    return count_mismatches, orig_sentence, sug_sentence

def correct_document_context_parallel_naive(fname, dictionary,
                             start_prob, default_start_prob,
                             transition_prob, default_transition_prob,
                             max_edit_distance=3, num_partitions=6,
                             display_results=False):
    
    '''
    Load a text file and spell-check each sentence using the
    dictionary and probability tables that were created in the
    pre-processing stage.

    Suggested corrections are either printed to the screen or
    saved in a log file, depending on the settings.
    '''

    # note: various partitioning strategies were attempted for this
    # function, but they failed to yield significant improvements in
    # performance at any file size.

    ############
    #
    # load file & initial processing
    #
    ############
    
    # http://stackoverflow.com/questions/22520932/python-remove-all-non-alphabet-chars-from-string
    regex = re.compile('[^a-z ]')

    # broadcast Python dictionaries to workers (from pre-processing)
    bc_dictionary = sc.broadcast(dictionary)
    bc_start_prob = sc.broadcast(start_prob)
    bc_transition_prob = sc.broadcast(transition_prob)
    
    # load file contents and convert into one long sequence of words
    # RDD format: 'line 1', 'line 2', 'line 3', ...
    make_all_lower = sc.textFile(fname) \
            .map(lambda line: line.lower()) \
            .filter(lambda x: x!='')
        
    # split into individual sentences and remove other punctuation
    # RDD format: [words of sentence 1], [words of sentence 2], ...
    # cache because this RDD is used in multiple operations 
    split_sentence = make_all_lower.flatMap(lambda 
        line: line.replace('?','.').replace('!','.').split('.')) \
             .map(lambda sentence: regex.sub(' ', sentence)) \
             .map(lambda sentence: sentence.split()) \
             .filter(lambda x: x!=[]).cache()
    
    # use accumulator to count the number of words checked
    accum_total_words = sc.accumulator(0)
    split_sentence.flatMap(lambda x: x) \
            .foreach(lambda x: accum_total_words.add(1))
    
    # assign a unique id to each sentence
    # RDD format: (0, [words of sentence1]), (1, [words of sentence2]), ...
    # cache here after completing transformations - results in 
    # improvements in runtime that scale with file size
    # partition as sentence id will remain the key going forward
    sentence_id = split_sentence.zipWithIndex().map(
        lambda (k, v): (v, k)).partitionBy(num_partitions).cache()

    ############
    #
    # spell-checking
    #
    ############

    # use map operation to apply Viterbi algorithm to each sentence
    # RDD format: (0, [original sentence1], [corrected sentence1]),
    #             (1, [original sentence2], [corrected sentence2]), ...
    sentence_correction = sentence_id.mapValues(lambda v: (v, 
                viterbi(v, bc_dictionary.value, bc_start_prob.value, 
                        default_start_prob, bc_transition_prob.value, 
                        default_transition_prob, max_edit_distance)))
    ############
    #
    # output results
    #
    ############
    
    # count the number of corrections per sentence and drop any
    # sentences without suggested corrections
    # RDD format: 
    # (0, (corrections, [original sentence1], [corrected sentence1])),
    # (1, (corrections, [original sentence2], [corrected sentence2])), ...
    sentence_errors = sentence_correction.mapValues(lambda v: 
        (get_count_mismatches(v))). \
            filter(lambda (k, v): v[0]>0)
    
    # collect all sentences with identified errors (as list)
    sentence_errors_list = sentence_errors.collect()
    
    # count the number of potentially misspelled words
    num_errors = sum([s[1][0] for s in sentence_errors_list])
    
    # print suggested corrections
    if display_results:
        for sentence in sentence_errors_list:
            print 'Sentence %i: %s --> %s' % (sentence[0],
                ' '.join(sentence[1][1]), ' '.join(sentence[1][2]))
            print '-----'
    
    # output suggested corrections to file
    else:
        f = open('spell-log.txt', 'w')
        for sentence in sentence_errors_list:
            f.write('Sentence %i: %s --> %s\n' % (sentence[0], 
                ' '.join(sentence[1][1]), ' '.join(sentence[1][2])))
        f.close()
    
    print '-----'
    print 'Total words checked: %i' % accum_total_words.value
    print 'Total potential errors found: %i' % num_errors

### Sample performance

In [ ]:
dictionary_file = 'testdata/big.txt'
check_file = 'testdata/yelp100reviews.txt'

In [ ]:
print 'Pre-processing with %s...' % dictionary_file

start_time = time.time()

dictionary, start_prob, default_start_prob, transition_prob, default_transition_prob = \
    parallel_create_dictionary(dictionary_file)

run_time = time.time() - start_time

print '-----'
print '%.2f seconds to run' % run_time
print '-----'

```
Pre-processing with testdata/big.txt...
Total words processed: 1105285
Total unique words in corpus: 29157
Total items in dictionary (corpus words and deletions): 2151998
  Edit distance for deletions: 3
Total unique words at the start of a sentence: 15356
Total unique word transitions: 27086
-----
54.25 seconds to run
-----
```

In [ ]:
print 'Spell-checking %s...' % check_file

start_time = time.time()

correct_document_context_parallel_naive(check_file, dictionary,
                        start_prob, default_start_prob, 
                        transition_prob, default_transition_prob)

run_time = time.time() - start_time

print '-----'
print '%.2f seconds to run' % run_time
print '-----'

```
Spell-checking testdata/yelp100reviews.txt...
-----
Total words checked: 12029
Total potential errors found: 1735
-----
196.14 seconds to run
-----
```
Sample output with suggested corrections <a href='sample/context-spell-log-yelp.txt'>here</a>.

### Notes on SPARK optimization

Various optimizations were attempted for this implementation, including modifications to partitioning and caching, as well as slight variations in the operations (e.g. replacing groupByKey with reduceByKey).  These changes all failed to yield significant improvements in performance; this is most likely due to the very simple nature of the algorithm, which only has one map operation related to the actual spell-checking.  The function correct_document_context_parallel_naive requires very minimal shuffling (~150KB shuffle reads and shuffles writes for 100 Yelp reviews), so there is little room for improvement in that respect.

It is worth noting the overhead associated with the pre-processing, which is necessary to create the lookup tables that are used by the spell-checker. In particular, creating the transition matrix involves two relatively large shuffle reads and shuffle writes (~8.6MB and ~4.5MB) associated with the reduceByKey and groupByKey operations. The creation of the dictionary and start word probabilities is considerably less expensive (~1MB shuffles for each).

We found that a relatively small number of partitions (6) performed consistently well across file sizes for both the pre-processing and spell-checking.

### Notes on local & AWS performance

The serial implementation consistently checks approximately 40 words per second. For smaller file sizes, this was clearly superior to the performance of the SPARK code, which was only able to check 4 words per second for 1 Yelp review, and 25 words per second for 10 Yelp reviews. However, the SPARK code began to outperform the serial code when moving to 100 Yelp reviews, achieving 61 words per second, and was able to reach 78 words per second when testing the file with 1,000 Yelp reviews. This file was the largest that could be run locally due to memory constraints.

Moving to AWS yielded further improvements. The smallest cluster tested (4 executors x 4 cores) was able to achieve 277 words per second on the same 1,000-review file - nearly 4 times the local speed. However, that particular cluster configuration would time out when attempting to run larger file sizes. We also experimented with different numbers of partitions on that cluster, and found that fewer partitions (16) performed slightly better than more partitions (64). Again, this is likely due to the simple nature of the implementation; increasing the number of partitions results in greater complexity without any offsetting gains.

The next step up was to move to a medium-size cluster (8 executors x 4 cores) and to increase the number of partitions accordingly to 64. This resulted in roughly a doubling in performance, with the same 1,000-review file being processed at 514 words per second. In addition, we were able to run the larger 10,000-review file, with even better performance of 585 words per second.

The last configuration that we tested was a 16 executor x 4 core cluster. This configuration was clearly most appropriate for files at the larger end of the spectrum: while it only achieved a 1.6x improvement for the 1,000-review file, it ran 2x faster for the 10,000-review file (over 1,100 words per second).

We had originally planned to test this implementation on even larger clusters with larger files, but were not able to do so due to the limit imposed by Amazon on the total number of instances.

We used m3.large instances for all of the above configurations, and upgraded the default AWS Python build to version 2.7 for consistency with our local development environment.

<img src='img/contextSPARK1performance.jpg' width=50% height=50%>

## SPARK Implementation # 2 - Approximate Parallelization
***

This implementation is inspired by the algorithm, but is not completely faithful to it. 

For a given sentence, the Viterbi algorithm steps through each word in turn. At each iteration, it assesses the combination of all previous paths and the word currently under consideration, and only stores the previous path with the highest probability. Consequently, only a subset of the possible combinations of words are retained by the last step.

In contrast, this implementation considers all the possible sentences that could be created through combinations of all possible corrections for all the words in a sentence.

e.g. "This is a test" -> if each word has 5 possible corrections, then there are 5^4 possible resulting sentences.

We calculate the probability of each sentence (using the same concepts of start probabilities, emission probabilities, and transition probabilities) and choose the sentence (word combination) with the highest probability.

There are two main problems with this implementation.

First, as expected, this approach yields similar corrections to the Viterbi algorithm in many cases, but not always. Despite not being a compeletely faithful representation of the original algorithm, this implementation was considered a way to experiment with a different way of parallelizing the problem.

The second problem is more severe. Under this implementation, the size of the problem grows exponentially with the size of the text being checked (e.g. an 8-word sentence results in 10^8 possible combinations). This not only makes the approach inefficient, but (more importantly) means that it does not scale to larger problems.

<img src='img/contextSPARK2.jpg' width=50% height=50%>

### Pre-processing

In [ ]:
######################
#
# PRE-PROCESSING STEPS
#
# The pre-processing steps have been adapted from the dictionary
# creation of the word-level spellchecker, which in turn was based on
# SymSpell, a Symmetric Delete spelling correction algorithm
# developed by Wolf Garbe and originally written in C#. More detail
# on SymSpell is included in the word-level spellcheck documentation.
#
# The main modifications to the word-level spellchecker pre-
# processing stages are to create the additional outputs that are
# required for the context-level checking, and to eliminate redundant
# outputs that are not necessary.
#
# The outputs of the pre-processing stage are:
#
# - dictionary: A dictionary that combines both words present in the
# corpus and other words that are within a given 'delete distance'. 
# The format of the dictionary is:
# {word: ([list of words within the given 'delete distance'], 
# word count in corpus)}
#
# - start_prob: A dictionary with key, value pairs that correspond to
# (word, probability of the word being the first word in a sentence)
#
# - transition_prob: A dictionary of dictionaries that stores the
# probability of a given word following another. The format of the
# dictionary is:
# {previous word: {word1 : P(word1|prevous word), word2 : 
# P(word2|prevous word), ...}}
#
# - default_start_prob: A benchmark probability of a word being at
# the start of a sentence, set to 1 / # of words at the beginning of
# sentences. This ensures that all previously unseen words at the
# beginning of sentences are not corrected unnecessarily.
#
# - default_transition_prob: A benchmark probability of a word being
# seen, given the previous word in the sentence, also set to 1 / # of
# transitions in corpus. This ensures that all previously unseen
# transitions are not corrected unnecessarily.
#
######################

def get_deletes_list(w, max_edit_distance):
    '''
    Given a word, derive strings with up to max_edit_distance
    characters deleted. 

    The list is generally of the same magnitude as the number of
    characters in a word, so it does not make sense to parallelize
    this function. Instead, we use Python to create the list.
    '''
    deletes = []
    queue = [w]
    for d in range(max_edit_distance):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes

def get_transitions(sentence):
    '''
    Helper function: converts a sentence into all two-word pairs.
    Output format is a list of tuples.
    e.g. 'This is a test' >> ('this', 'is'), ('is', 'a'), ('a', 'test')
    ''' 
    if len(sentence)<2:
        return None
    else:
        return [((sentence[i], sentence[i+1]), 1) 
                for i in range(len(sentence)-1)]
    
def map_transition_prob(vals):
    '''
    Helper function: calculates conditional probabilities for all word
    pairs, i.e. P(word|previous word)
    '''
    total = float(sum(vals.values()))
    return {k: math.log(v/total) for k, v in vals.items()}

def parallel_create_dictionary(fname, max_edit_distance=3, 
                                num_partitions=6):
    '''
    Load a text file and use it to create a dictionary and
    to calculate start probabilities and transition probabilities. 
    '''
    
    # Note: this function makes use of multiple accumulators to keep
    # track of the words that are being processed. An alternative 
    # implementation that wraps accumulators in helper functions was
    # also tested, but did not yield any noticeable improvements.

    ############
    #
    # load file & initial processing
    #
    ############
    
    # http://stackoverflow.com/questions/22520932/python-remove-all-non-alphabet-chars-from-string
    regex = re.compile('[^a-z ]')

    # load file contents and convert into one long sequence of words
    # RDD format: 'line 1', 'line 2', 'line 3', ...
    # cache because this RDD is used in multiple operations 
    make_all_lower = sc.textFile(fname) \
            .map(lambda line: line.lower()) \
            .filter(lambda x: x!='').cache()
    
    # split into individual sentences and remove other punctuation
    # RDD format: [words of sentence 1], [words of sentence 2], ...
    # cache because this RDD is used in multiple operations 
    split_sentence = make_all_lower.flatMap(lambda 
        line: line.replace('?','.').replace('!','.').split('.')) \
             .map(lambda sentence: regex.sub(' ', sentence)) \
             .map(lambda sentence: sentence.split()) \
             .filter(lambda x: x!=[]).cache()
    
    ############
    #
    # generate start probabilities
    #
    ############
    
    # extract all words that are at the beginning of sentences
    # RDD format: 'word1', 'word2', 'word3', ...
    start_words = split_sentence.map(lambda sentence: sentence[0] 
        if len(sentence)>0 else None) \
            .filter(lambda word: word!=None)
    
    # add a count to each word
    # RDD format: ('word1', 1), ('word2', 1), ('word3', 1), ...
    # note: partition here because we are using words as keys for
    # the first time - yields a small but consistent improvement in
    # runtime (~2-3 sec for big.txt)
    # cache because this RDD is used in multiple operations
    count_start_words_once = start_words.map(lambda word: (word, 1)) \
            .partitionBy(num_partitions).cache()

    # use accumulator to count the number of start words processed
    accum_total_start_words = sc.accumulator(0)
    count_start_words_once.foreach(lambda x: accum_total_start_words.add(1))
    total_start_words = float(accum_total_start_words.value)
    
    # reduce into count of unique words at the start of sentences
    # RDD format: ('word1', frequency), ('word2', frequency), ...
    unique_start_words = count_start_words_once.reduceByKey(lambda a, b: a + b)
    
    # convert counts to log-probabilities
    # RDD format: ('word1', log-prob of word1), 
    #             ('word2', log-prob of word2), ...
    start_prob_calc = unique_start_words.mapValues(lambda v: 
        math.log(v/total_start_words))
    
    # get default start probabilities (for words not in corpus)
    default_start_prob = math.log(1/total_start_words)
    
    # store start probabilities as a dictionary (i.e. a lookup table)
    # note: given the spell-checking algorithm, this cannot be maintained
    # as an RDD as it is not possible to map within a map
    start_prob = start_prob_calc.collectAsMap()
    
    ############
    #
    # generate transition probabilities
    #
    ############
    
    # note: various partitioning strategies were attempted for this
    # portion of the function, but they failed to yield significant
    # improvements in performance.

    # focus on continuous word pairs within the sentence
    # e.g. "this is a test" -> "this is", "is a", "a test"
    # note: as the relevant probability is P(word|previous word)
    # the tuples are ordered as (previous word, word)

    # extract all word pairs within a sentence and add a count
    # RDD format: (('word1', 'word2'), 1), (('word2', 'word3'), 1), ...
    # cache because this RDD is used in multiple operations 
    other_words = split_sentence.map(lambda sentence: 
        get_transitions(sentence)) \
            .filter(lambda x: x!=None) \
            .flatMap(lambda x: x).cache()

    # use accumulator to count the number of transitions (word pairs)
    accum_total_other_words = sc.accumulator(0)
    other_words.foreach(lambda x: accum_total_other_words.add(1))
    total_other_words = float(accum_total_other_words.value)
    
    # reduce into count of unique word pairs
    # RDD format: (('word1', 'word2'), frequency), 
    #             (('word2', 'word3'), frequency), ...
    unique_other_words = other_words.reduceByKey(lambda a, b: a + b)
    
    # aggregate by (and change key to) previous word
    # RDD format: ('previous word', {'word1': word pair count, 
    #                                'word2': word pair count}}), ...
    other_words_collapsed = unique_other_words.map(lambda x: 
        (x[0][0], (x[0][1], x[1]))) \
            .groupByKey().mapValues(dict)

    # note: the above line of code is the slowest in the function
    # (8.6 MB shuffle read and 4.5 MB shuffle write for big.txt)
    # An alternative approach that aggregates lists with reduceByKey was
    # attempted, but did not yield noticeable improvements in runtime.
    
    # convert counts to log-probabilities
    # RDD format: ('previous word', {'word1': log-prob of pair, 
    #                                 word2: log-prob of pair}}), ...
    transition_prob_calc = other_words_collapsed.mapValues(lambda v: 
        map_transition_prob(v))

    # get default transition probabilities (for word pairs not in corpus)
    default_transition_prob = math.log(1/total_other_words)
    
    # store transition probabilities as a dictionary (i.e. a lookup table)
    # note: given the spell-checking algorithm, this cannot be maintained
    # as an RDD as it is not possible to map within a map
    transition_prob = transition_prob_calc.collectAsMap()
    
    ############
    #
    # generate dictionary
    #
    ############

    # note: this approach is slightly different from the original SymSpell
    # algorithm, but is more appropriate for a SPARK implementation
    
    # split into individual words (all)
    # RDD format: 'word1', 'word2', 'word3', ...
    # cache because this RDD is used in multiple operations 
    all_words = make_all_lower.map(lambda line: regex.sub(' ', line)) \
            .flatMap(lambda line: line.split()).cache()

    # use accumulator to count the number of words processed
    accum_words_processed = sc.accumulator(0)
    all_words.foreach(lambda x: accum_words_processed.add(1))

    # add a count to each word
    # RDD format: ('word1', 1), ('word2', 1), ('word3', 1), ...
    count_once = all_words.map(lambda word: (word, 1))

    # reduce into counts of unique words - this is the core corpus dictionary
    # (i.e. only words appearing in the file, without 'deletes'))
    # RDD format: ('word1', frequency), ('word2', frequency), ...
    # cache because this RDD is used in multiple operations 
    # note: imposing partitioning at this step yields a small 
    # improvement in runtime (~1 sec for big.txt) by equally
    # balancing elements among workers for subsequent operations
    unique_words_with_count = count_once.reduceByKey(lambda a, b: a + b, 
        numPartitions = num_partitions).cache()
    
    # use accumulator to count the number of unique words
    accum_unique_words = sc.accumulator(0)
    unique_words_with_count.foreach(lambda x: accum_unique_words.add(1))

    # generate list of "deletes" for each word in the corpus
    # RDD format: (word1, [deletes for word1]), (word2, [deletes for word2]), ...
    generate_deletes = unique_words_with_count.map(lambda (parent, count): 
        (parent, get_deletes_list(parent, max_edit_distance)))
    
    # split into all key-value pairs
    # RDD format: (word1, delete1), (word1, delete2), ...
    expand_deletes = generate_deletes.flatMapValues(lambda x: x)
    
    # swap word order and add a zero count (because "deletes" were not
    # present in the dictionary)
    swap = expand_deletes.map(lambda (orig, delete): (delete, ([orig], 0)))
    
    # create a placeholder for each real word
    # RDD format: ('word1', ([], frequency)), ('word2', ([], frequency)), ...
    corpus = unique_words_with_count.mapValues(lambda count: ([], count))

    # combine main dictionary and "deletes" (and eliminate duplicates)
    # RDD format: ('word1', ([deletes for word1], frequency)), 
    #             ('word2', ([deletes for word2], frequency)), ...
    combine = swap.union(corpus)
    
    # store dictionary items and deletes as a dictionary (i.e. a lookup table)
    # note: given the spell-checking algorithm, this cannot be maintained
    # as an RDD as it is not possible to map within a map
    # note: use reduceByKeyLocally to avoid an extra shuffle from reduceByKey
    dictionary = combine.reduceByKeyLocally(lambda a, b: (a[0]+b[0], a[1]+b[1])) 
    
    # output stats
    print 'Total words processed: %i' % accum_words_processed.value
    print 'Total unique words in corpus: %i' % accum_unique_words.value 
    print 'Total items in dictionary (corpus words and deletions): %i' \
        % len(dictionary)
    print '  Edit distance for deletions: %i' % max_edit_distance
    print 'Total unique words at the start of a sentence: %i' \
        % len(start_prob)
    print 'Total unique word transitions: %i' % len(transition_prob)
    
    return dictionary, start_prob, default_start_prob, \
            transition_prob, default_transition_prob

### Document correction

In [ ]:
######################
#
# SPELL-CHECKING - (APPROXIMATE) VITERBI ALGORITHM
#
# The below functions are used to read in a text file, break it down
# into individual sentences, and then carry out context-based spell-
# checking on each sentence in turn. In cases where the 'suggested'
# word does not match the actual word in the text, both the original
# and the suggested sentences are printed/outputed to file.
#
# Probabilistic model:
#
# Each sentence is modeled as a hidden Markov model, where the
# hidden states are the words that the user intended to type, and
# the emissions are the words that were actually typed.
#
# For each word in a sentence, we can define:
#
# - emission probabilities: P(observed word|intended word)
#
# - prior probabilities (for first words in sentences only):
# P(being the first word in a sentence)
#
# - transition probabilities (for all subsequent words):
# P(intended word|previous intended word)
#
# Prior and transition probabilities were calculated in the pre-
# processing steps above, using the same corpus as the dictionary.
# 
# Emission probabilities are calculated on the fly using a Poisson
# distribution as follows:
# P(observed word|intended word) = PMF of Poisson(k, l), where
# k = edit distance between word typed and word intended, and l=0.01.
# Both the overall approach and the parameter of l=0.01 are based on
# the 2015 lecture notes from AM207 Stochastic Optimization.
# Various parameters for lambda between 0 and 1 were tested, which
# confirmed that 0.01 yields the most accurate word suggestions.
#
# All probabilities are stored in log-space to avoid underflow. Pre-
# defined minimum values (also defined at the pre-processing stage)
# are used for words that are not present in the dictionary and/or
# probability tables.
#
# Algorithm:
#
# The spell-checking itself is carried out using a modified version
# of the Viterbi algorithm, which yields the most likely sequence of
# hidden states, i.e. the most likely sequence of words that form a
# sentence. As with the other implementations, the state space (i.e.
# list of possible corrections) is generated (and therefore varies)
# for each word. This is in contrast to the alternative of considering
# the state space of all possible words in the dictionary for every
# word that is checked, which would be intractable for larger
# dictionaries.
#
# This version also uses a further approximation to the original
# algorithm.
#
# For a given sentence, the Viterbi algorithm steps through each word
# in turn. At each iteration, it assesses the combination of all
# previous paths and the word currently under consideration, and only
# stores the previous path with the highest probability. Consequently,
# only a subset of the possible combinations of words are retained
# by the last step.
#
# In contrast, this implementation considers all the possible
# sentences that could be created through combinations of all possible
# corrections for all the words in a sentence.
# e.g. "This is a test" -> if each word has 5 possible corrections,
# then there are 5^4 possible resulting sentences.
# We calculate the probability of each sentence (using the same concepts
# of start probabilities, emission probabilities, and transition
# probabilities) and choose the sentence (word combination) with the
# highest probability.
#
# As expected, we found that this approach yielded similar corrections
# in many cases, but not always. Despite not being a faithful
# representation of the original algorithm, this implementation allowed
# us to experiment with a different way of parallelizing the problem
# of context-level checking.
#
######################

def dameraulevenshtein(seq1, seq2):
    '''
    Calculate the Damerau-Levenshtein distance between sequences.

    codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1
    matrix. However, only the current and two previous rows are
    needed at once, so we only store those.

    Same code as word-level checking.
    '''
    
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    
    for x in xrange(len(seq1)):
        
        twoago, oneago, thisrow = \
            oneago, thisrow, [0] * len(seq2) + [x + 1]
        
        for y in xrange(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)

            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
                
    return thisrow[len(seq2) - 1]

def get_suggestions(string, dictionary, max_edit_distance, 
                    longest_word_length=20, min_count=100, max_sug=10):
    '''
    Return list of suggested corrections for potentially incorrectly
    spelled word.

    Code based on get_suggestions function from word-level checking,
    with the addition of the min_count and max_sug parameters.
    - min_count: minimum number of times a word must have appeared
    in the dictionary corpus to be considered a valid suggestion
    - max_sug: number of suggestions that are returned (ranked by
    frequency of appearance in dictionary corpus and edit distance
    from word being checked)

    These changes were imposed in order to ensure that the problem
    remains tractable when checking very large documents. In practice,
    the "correct" suggestion is almost always amongst the top ten.

    '''
    
    if (len(string) - longest_word_length) > max_edit_distance:
        # to ensure Viterbi can keep running -- use the word itself
        return [(string, 0)]
    
    suggest_dict = {}
    
    queue = [string]
    q_dictionary = {}  # items other than string that we've checked
    
    while len(queue)>0:
        q_item = queue[0]  # pop
        queue = queue[1:]
        
        # process queue item
        if (q_item in dictionary) and (q_item not in suggest_dict):
            if (dictionary[q_item][1]>0):
            # word is in dictionary, and is a word from the corpus,
            # and not already in suggestion list so add to suggestion
            # dictionary, indexed by the word with value (frequency
            # in corpus, edit distance)
            # note: q_items that are not the input string are shorter
            # than input string since only deletes are added (unless
            # manual dictionary corrections are added)
                assert len(string)>=len(q_item)
                suggest_dict[q_item] = \
                    (dictionary[q_item][1], len(string) - len(q_item))
            
            # the suggested corrections for q_item as stored in
            # dictionary (whether or not q_item itself is a valid
            # word or merely a delete) can be valid corrections
            for sc_item in dictionary[q_item][0]:
                if (sc_item not in suggest_dict):
                    
                    # compute edit distance
                    # suggested items should always be longer (unless
                    # manual corrections are added)
                    assert len(sc_item)>len(q_item)
                    # q_items that are not input should be shorter
                    # than original string 
                    # (unless manual corrections added)
                    assert len(q_item)<=len(string)
                    if len(q_item)==len(string):
                        assert q_item==string
                        item_dist = len(sc_item) - len(q_item)

                    # item in suggestions list should not be the same
                    # as the string itself
                    assert sc_item!=string           
                    # calculate edit distance using Damerau-
                    # Levenshtein distance
                    item_dist = dameraulevenshtein(sc_item, string)
                    
                    if item_dist<=max_edit_distance:
                        # should already be in dictionary if in
                        # suggestion list
                        assert sc_item in dictionary  
                        # trim list to contain state space
                        if (dictionary[q_item][1]>0): 
                            suggest_dict[sc_item] = \
                                (dictionary[sc_item][1], item_dist)
        
        # now generate deletes (e.g. a substring of string or of a
        # delete) from the queue item as additional items to check
        # -- add to end of queue
        assert len(string)>=len(q_item)
        if (len(string)-len(q_item))<max_edit_distance \
            and len(q_item)>1:
            for c in range(len(q_item)): # character index        
                word_minus_c = q_item[:c] + q_item[c+1:]
                if word_minus_c not in q_dictionary:
                    queue.append(word_minus_c)
                    # arbitrary value to identify we checked this
                    q_dictionary[word_minus_c] = None

    # return list of suggestions: (correction, edit distance)
    
    # only include words that have appeared a minimum number of times
    # note: make sure that we do not lose the original word
    as_list = [i for i in suggest_dict.items() 
               if (i[1][0]>min_count or i[0]==string)]
    
    # only include the most likely suggestions (based on frequency
    # and edit distance from original word)
    trunc_as_list = sorted(as_list, 
            key = lambda (term, (freq, dist)): (dist, -freq))[:max_sug]
    
    if len(trunc_as_list)==0:
        # to ensure Viterbi can keep running
        # -- use the word itself if no corrections are found
        return [(string, 0)]
        
    else:
        # drop the word frequency - not needed beyond this point
        return [(i[0], i[1][1]) for i in trunc_as_list]

    '''
    Output format:
    get_suggestions('file', dictionary)
    [('file', 0), ('five', 1), ('fire', 1), ('fine', 1), ('will', 2),
    ('time', 2), ('face', 2), ('like', 2), ('life', 2), ('while', 2)]
    '''
    
def get_emission_prob(edit_dist, poisson_lambda=0.01):
    '''
    The emission probability, i.e. P(observed word|intended word)
    is approximated by a Poisson(k, l) distribution, where 
    k=edit distance between the observed word and the intended
    word and l=0.01.
    
    Both the overall approach and the parameter of l=0.01 are based on
    the 2015 lecture notes from AM207 Stochastic Optimization.
    Various parameters for lambda between 0 and 1 were tested, which
    confirmed that 0.01 yields the most accurate word suggestions.
    '''
    
    return math.log(poisson.pmf(edit_dist, poisson_lambda))

######################
# Multiple helper functions are used to avoid KeyErrors when
# attempting to access values that are not present in dictionaries,
# in which case the previously specified default value is returned.
######################

def get_start_prob(word, start_prob, default_start_prob):
    '''
    P(word being at the beginning of a sentence)
    '''
    try:
        return start_prob[word]
    except KeyError:
        return default_start_prob
    
def get_transition_prob(cur_word, prev_word, 
                        transition_prob, default_transition_prob):
    '''
    P(word|previous word)
    '''
    try:
        return transition_prob[prev_word][cur_word]
    except KeyError:
        return default_transition_prob

def map_sentence_words(sentence, tmp_dict, max_edit_distance):
    '''
    Helper function: returns all suggestions for all words
    in a sentence.
    '''
    return [[word, get_suggestions(word, tmp_dict, max_edit_distance)] 
            for i, word in enumerate(sentence)]

def split_suggestions_app(sentence):
    '''
    Helper function: create word-suggestion pairs for each
    word in the sentence, and look up the emission probability
    of each pair.
    '''
    result = []
    for word in sentence:
        result.append([(word[0], s[0], get_emission_prob(s[1])) 
                       for s in word[1]])
    return result

def get_word_combos(sug_lists):
    '''
    Helper function: returns all possible sentences that can be
    created from the list of suggestions for each word in the
    original sentence.
    e.g. a sentence with 5 words and 10 suggestions per word
    will result in 10^6 possible sentences.
    '''
    return list(itertools.product(*sug_lists))

def get_combo_prob(combo, tmp_sp, d_sp, tmp_tp, d_tp):
    
    # first word in sentence
    # emission prob * start prob
    orig_path = [combo[0][0]]
    sug_path = [combo[0][1]]
    prob = combo[0][2] + get_start_prob(combo[0][1], tmp_sp, d_sp)
    
    # subsequent words
    for i, w in enumerate(combo[1:]):
        orig_path.append(w[0])
        sug_path.append(w[1])
        prob += w[2] + get_transition_prob(w[1], combo[i-1][1], tmp_tp, d_tp)
    
    return orig_path, sug_path, prob

def get_count_mismatches(sentences):
    '''
    Helper function: compares the original sentence with the sentence
    that has been suggested by the Viterbi algorithm, and calculates
    the number of words that do not match.
    '''
    orig_sentence, sug_sentence, prob = sentences
    count_mismatches = len([(orig_sentence[i], sug_sentence[i]) 
            for i in range(len(orig_sentence))
            if orig_sentence[i]!=sug_sentence[i]])
    return count_mismatches, orig_sentence, sug_sentence

def correct_document_context_parallel_approximate(fname, dictionary,
                             start_prob, default_start_prob,
                             transition_prob, default_transition_prob,
                             max_edit_distance=3, num_partitions=6,
                             display_results=False):
    
    '''
    Load a text file and spell-check each sentence using the
    dictionary and probability tables that were created in the
    pre-processing stage.

    Suggested corrections are either printed to the screen or
    saved in a log file, depending on the settings.
    '''

    ############
    #
    # load file & initial processing
    #
    ############
    
    # http://stackoverflow.com/questions/22520932/python-remove-all-non-alphabet-chars-from-string
    regex = re.compile('[^a-z ]')
    
    # broadcast Python dictionaries to workers (from pre-processing)
    bc_dictionary = sc.broadcast(dictionary)
    bc_start_prob = sc.broadcast(start_prob)
    bc_transition_prob = sc.broadcast(transition_prob)
    
    # load file contents and convert into one long sequence of words
    # RDD format: 'line 1', 'line 2', 'line 3', ...
    make_all_lower = sc.textFile(fname) \
            .map(lambda line: line.lower()) \
            .filter(lambda x: x!='')
    
    # split into individual sentences and remove other punctuation
    # RDD format: [words of sentence 1], [words of sentence 2], ...
    # cache because this RDD is used in multiple operations 
    split_sentence = make_all_lower.flatMap(lambda 
        line: line.replace('?','.').replace('!','.').split('.')) \
             .map(lambda sentence: regex.sub(' ', sentence)) \
             .map(lambda sentence: sentence.split()) \
             .filter(lambda x: x!=[]).cache()
    
    # use accumulator to count the number of words checked
    accum_total_words = sc.accumulator(0)
    split_sentence.flatMap(lambda x: x) \
            .foreach(lambda x: accum_total_words.add(1))
    
    # assign a unique id to each sentence
    # RDD format: (0, [words of sentence1]), (1, [words of sentence2]), ...
    # cache here after completing transformations - results in 
    # improvements in runtime that scale with file size
    # partition as sentence id will remain the key going forward
    sentence_id = split_sentence.zipWithIndex().map(
        lambda (k, v): (v, k)).cache()
    
    ############
    #
    # spell-checking
    #
    ############
    
    # look up possible suggestions for each word in each sentence
    # RDD format:
    # (sentence id, [[word, [suggestions]], [word, [suggestions]], ... ]),
    # (sentence id, [[word, [suggestions]], [word, [suggestions]], ... ]), ...
    sentence_words = sentence_id.mapValues(lambda v: 
        map_sentence_words(v, bc_dictionary.value, max_edit_distance))
    
    # look up emission probabilities for each word
    # i.e. P(observed word|intended word)
    # RDD format: (one list of tuples per word in the sentence)
    # (sentence id, [(original word, suggested word, P(original|suggested)),
    #                (original word, suggested word, P(original|suggested)), ...]), ...
    sentence_word_sug = sentence_words.mapValues(lambda v: 
        split_suggestions_app(v))
    
    # generate all possible corrected combinations (using Cartesian
    # product) - i.e. a sentence with 4 word, each of which have 5 
    # possible suggestions, will yield 5^4 possible combinations
    # RDD format: (a tuple of tuples for each combination)
    # (sentence id, [(original word1, suggested word1, P(original1|suggested1)),
    #                (original word2, suggested word2, P(original2|suggested2)), ...]), ...
    sentence_word_combos = sentence_word_sug.mapValues(lambda v:
        get_word_combos(v))
    
    # flatmap into all possible combinations per sentence
    # RDD format: (a tuple of tuples for one combination)
    # (sentence id, [(original word1, suggested word1, P(original1|suggested1)),
    #                (original word2, suggested word2, P(original2|suggested2)), ...]), ...
    # partitioning after the flatmap results in a small improvement in runtime
    # for the smaller texts on which we were able to run the function
    sentence_word_combos_split = sentence_word_combos.flatMapValues(lambda x: x) \
            .partitionBy(num_partitions).cache()
    
    # calculate the probability of each word combination being the
    # intended one, given what was actually typed
    # note: the approach does not drop any word combinations, so may
    # yield different results to the Viterbi algorithm
    # RDD format: 
    # (sentence id, ([original sentence], [suggested sentence], probability)),
    # (sentence id, ([original sentence], [suggested sentence], probability)), ...
    sentence_word_combos_prob = sentence_word_combos_split.mapValues(
        lambda v: get_combo_prob(v, bc_start_prob.value, default_start_prob, 
            bc_transition_prob.value, default_transition_prob))
    
    # identify the word combination with the highest probability for
    # each sentence
    # (sentence id, ([original sentence], [suggested sentence], probability)),
    # (sentence id, ([original sentence], [suggested sentence], probability)), ...
    sentence_max_prob = sentence_word_combos_prob.reduceByKey(
        lambda a,b: a if a[2] > b[2] else b)

    ############
    #
    # output results
    #
    ############

    # count the number of errors per sentence and drop any sentences
    # without errors
    # RDD format: (sentence id, (# errors, [original sentence], [suggested sentence])),
    #             (sentence id, (# errors, [original sentence], [suggested sentence])), ...
    sentence_errors = sentence_max_prob.mapValues(
        lambda v: get_count_mismatches(v)) \
            .filter(lambda (k, v): v[0]>0)

    # collect all sentences with identified errors (as list)
    sentence_errors_list = sentence_errors.collect()
    
    # count the number of potentially misspelled words
    num_errors = sum([s[1][0] for s in sentence_errors_list])
    
    # print suggested corrections
    if display_results:
        for sentence in sentence_errors_list:
            print 'Sentence %i: %s --> %s' % (sentence[0],
                ' '.join(sentence[1][1]), ' '.join(sentence[1][2]))
            print '-----'
    
    # output suggested corrections to file
    else:
        f = open('spell-log.txt', 'w')
        for sentence in sentence_errors_list:
            f.write('Sentence %i: %s --> %s\n' % (sentence[0], 
                ' '.join(sentence[1][1]), ' '.join(sentence[1][2])))
        f.close()
    
    print '-----'
    print 'Total words checked: %i' % accum_total_words.value
    print 'Total potential errors found: %i' % num_errors

### Sample performance

<div class="alert alert-danger">
<strong>WARNING:</strong> The memory required by this implementation grows exponentially with the size of the problem. Do not attempt to run this code for larger files.
</div>

In [ ]:
dictionary_file = 'testdata/big.txt'
check_file = 'testdata/test.txt'

In [ ]:
print 'Pre-processing with %s...' % dictionary_file

start_time = time.time()

dictionary, start_prob, default_start_prob, transition_prob, default_transition_prob = \
    parallel_create_dictionary(dictionary_file)

run_time = time.time() - start_time

print '-----'
print '%.2f seconds to run' % run_time
print '-----'

```
Pre-processing with testdata/big.txt...
Total words processed: 1105285
Total unique words in corpus: 29157
Total items in dictionary (corpus words and deletions): 2151998
  Edit distance for deletions: 3
Total unique words at the start of a sentence: 15356
Total unique word transitions: 27086
-----
54.25 seconds to run
-----
```

In [ ]:
print 'Spell-checking %s...' % check_file

start_time = time.time()

correct_document_context_parallel_approximate(check_file, dictionary,
                        start_prob, default_start_prob, 
                        transition_prob, default_transition_prob)

run_time = time.time() - start_time

print '-----'
print '%.2f seconds to run' % run_time
print '-----'

```
Spell-checking testdata/test.txt...
-----
Total words checked: 27
Total potential errors found: 4
-----
24.99 seconds to run
-----
```
Sample output with suggested corrections <a href='sample/context-spell-log-test.txt'>here</a>.   
Note: the variations in the implementation result in different suggested corrections, compared to the previous or the next implementation. In particular, the running example of "this is ax test" is not correctly updated.

### Notes on SPARK optimization
This implementation was originally developed and tested on a very small test file (test.txt). It became apparent when moving to larger files that the memory required by this approach grows exponentially with the size of the problem, making it very badly suited for any kind of scaling.

For this reason, this particular implementation was not taken any further but is included here for completeness.

## SPARK Implementation # 3 - Full Parallelization
***
Unlike previous implementations that relied heavily on helper functions to assess individual sentences, in this version we implement all the steps of the Viterbi algorithm in SPARK.

The algorithm breaks each sentence to be corrected into its word components, and then loops through each word position exactly as in the original serial algorithm.

Because different sentences in the document may have different lengths but are all being processed in parallel, we check at each iteration for any sentences that are "done" and store their results for later use. Once the maximum sentence length is reached, the results for all of the sentences are processed to extract and display/print suggested corrections.

This approach takes advantage of parallelization by splitting all the words at a given position among the workers.

<img src='img/contextSPARK3.jpg' width=50% height=50%>

### Pre-processing

In [ ]:
######################
#
# PRE-PROCESSING STEPS
#
# The pre-processing steps have been adapted from the dictionary
# creation of the word-level spellchecker, which in turn was based on
# SymSpell, a Symmetric Delete spelling correction algorithm
# developed by Wolf Garbe and originally written in C#. More detail
# on SymSpell is included in the word-level spellcheck documentation.
#
# The main modifications to the word-level spellchecker pre-
# processing stages are to create the additional outputs that are
# required for the context-level checking, and to eliminate redundant
# outputs that are not necessary.
#
# The outputs of the pre-processing stage are:
#
# - dictionary: A dictionary that combines both words present in the
# corpus and other words that are within a given 'delete distance'. 
# The format of the dictionary is:
# {word: ([list of words within the given 'delete distance'], 
# word count in corpus)}
#
# - start_prob: A dictionary with key, value pairs that correspond to
# (word, probability of the word being the first word in a sentence)
#
# - transition_prob: A dictionary of dictionaries that stores the
# probability of a given word following another. The format of the
# dictionary is:
# {previous word: {word1 : P(word1|prevous word), word2 : 
# P(word2|prevous word), ...}}
#
# - default_start_prob: A benchmark probability of a word being at
# the start of a sentence, set to 1 / # of words at the beginning of
# sentences. This ensures that all previously unseen words at the
# beginning of sentences are not corrected unnecessarily.
#
# - default_transition_prob: A benchmark probability of a word being
# seen, given the previous word in the sentence, also set to 1 / # of
# transitions in corpus. This ensures that all previously unseen
# transitions are not corrected unnecessarily.
#
######################

def get_deletes_list(w, max_edit_distance):
    '''
    Given a word, derive strings with up to max_edit_distance
    characters deleted. 

    The list is generally of the same magnitude as the number of
    characters in a word, so it does not make sense to parallelize
    this function. Instead, we use Python to create the list.
    '''
    deletes = []
    queue = [w]
    for d in range(max_edit_distance):
        temp_queue = []
        for word in queue:
            if len(word)>1:
                for c in range(len(word)):  # character index
                    word_minus_c = word[:c] + word[c+1:]
                    if word_minus_c not in deletes:
                        deletes.append(word_minus_c)
                    if word_minus_c not in temp_queue:
                        temp_queue.append(word_minus_c)
        queue = temp_queue
        
    return deletes

def get_transitions(sentence):
    '''
    Helper function: converts a sentence into all two-word pairs.
    Output format is a list of tuples.
    e.g. 'This is a test' >> ('this', 'is'), ('is', 'a'), ('a', 'test')
    ''' 
    if len(sentence)<2:
        return None
    else:
        return [((sentence[i], sentence[i+1]), 1) 
                for i in range(len(sentence)-1)]
    
def map_transition_prob(vals):
    '''
    Helper function: calculates conditional probabilities for all word
    pairs, i.e. P(word|previous word)
    '''
    total = float(sum(vals.values()))
    return {k: math.log(v/total) for k, v in vals.items()}

def parallel_create_dictionary(fname, max_edit_distance=3, 
                                num_partitions=6):
    '''
    Load a text file and use it to create a dictionary and
    to calculate start probabilities and transition probabilities. 
    '''
    
    # Note: this function makes use of multiple accumulators to keep
    # track of the words that are being processed. An alternative 
    # implementation that wraps accumulators in helper functions was
    # also tested, but did not yield any noticeable improvements.

    ############
    #
    # load file & initial processing
    #
    ############
    
    # http://stackoverflow.com/questions/22520932/python-remove-all-non-alphabet-chars-from-string
    regex = re.compile('[^a-z ]')

    # load file contents and convert into one long sequence of words
    # RDD format: 'line 1', 'line 2', 'line 3', ...
    # cache because this RDD is used in multiple operations 
    make_all_lower = sc.textFile(fname) \
            .map(lambda line: line.lower()) \
            .filter(lambda x: x!='').cache()
    
    # split into individual sentences and remove other punctuation
    # RDD format: [words of sentence 1], [words of sentence 2], ...
    # cache because this RDD is used in multiple operations 
    split_sentence = make_all_lower.flatMap(lambda 
        line: line.replace('?','.').replace('!','.').split('.')) \
             .map(lambda sentence: regex.sub(' ', sentence)) \
             .map(lambda sentence: sentence.split()) \
             .filter(lambda x: x!=[]).cache()
    
    ############
    #
    # generate start probabilities
    #
    ############
    
    # extract all words that are at the beginning of sentences
    # RDD format: 'word1', 'word2', 'word3', ...
    start_words = split_sentence.map(lambda sentence: sentence[0] 
        if len(sentence)>0 else None) \
            .filter(lambda word: word!=None)
    
    # add a count to each word
    # RDD format: ('word1', 1), ('word2', 1), ('word3', 1), ...
    # note: partition here because we are using words as keys for
    # the first time - yields a small but consistent improvement in
    # runtime (~2-3 sec for big.txt)
    # cache because this RDD is used in multiple operations
    count_start_words_once = start_words.map(lambda word: (word, 1)) \
            .partitionBy(num_partitions).cache()

    # use accumulator to count the number of start words processed
    accum_total_start_words = sc.accumulator(0)
    count_start_words_once.foreach(lambda x: accum_total_start_words.add(1))
    total_start_words = float(accum_total_start_words.value)
    
    # reduce into count of unique words at the start of sentences
    # RDD format: ('word1', frequency), ('word2', frequency), ...
    unique_start_words = count_start_words_once.reduceByKey(lambda a, b: a + b)
    
    # convert counts to log-probabilities
    # RDD format: ('word1', log-prob of word1), 
    #             ('word2', log-prob of word2), ...
    start_prob_calc = unique_start_words.mapValues(lambda v: 
        math.log(v/total_start_words))
    
    # get default start probabilities (for words not in corpus)
    default_start_prob = math.log(1/total_start_words)
    
    # store start probabilities as a dictionary (i.e. a lookup table)
    # note: given the spell-checking algorithm, this cannot be maintained
    # as an RDD as it is not possible to map within a map
    start_prob = start_prob_calc.collectAsMap()
    
    ############
    #
    # generate transition probabilities
    #
    ############
    
    # note: various partitioning strategies were attempted for this
    # portion of the function, but they failed to yield significant
    # improvements in performance.

    # focus on continuous word pairs within the sentence
    # e.g. "this is a test" -> "this is", "is a", "a test"
    # note: as the relevant probability is P(word|previous word)
    # the tuples are ordered as (previous word, word)

    # extract all word pairs within a sentence and add a count
    # RDD format: (('word1', 'word2'), 1), (('word2', 'word3'), 1), ...
    # cache because this RDD is used in multiple operations 
    other_words = split_sentence.map(lambda sentence: 
        get_transitions(sentence)) \
            .filter(lambda x: x!=None) \
            .flatMap(lambda x: x).cache()

    # use accumulator to count the number of transitions (word pairs)
    accum_total_other_words = sc.accumulator(0)
    other_words.foreach(lambda x: accum_total_other_words.add(1))
    total_other_words = float(accum_total_other_words.value)
    
    # reduce into count of unique word pairs
    # RDD format: (('word1', 'word2'), frequency), 
    #             (('word2', 'word3'), frequency), ...
    unique_other_words = other_words.reduceByKey(lambda a, b: a + b)
    
    # aggregate by (and change key to) previous word
    # RDD format: ('previous word', {'word1': word pair count, 
    #                                'word2': word pair count}}), ...
    other_words_collapsed = unique_other_words.map(lambda x: 
        (x[0][0], (x[0][1], x[1]))) \
            .groupByKey().mapValues(dict)

    # note: the above line of code is the slowest in the function
    # (8.6 MB shuffle read and 4.5 MB shuffle write for big.txt)
    # An alternative approach that aggregates lists with reduceByKey was
    # attempted, but did not yield noticeable improvements in runtime.
    
    # convert counts to log-probabilities
    # RDD format: ('previous word', {'word1': log-prob of pair, 
    #                                 word2: log-prob of pair}}), ...
    transition_prob_calc = other_words_collapsed.mapValues(lambda v: 
        map_transition_prob(v))

    # get default transition probabilities (for word pairs not in corpus)
    default_transition_prob = math.log(1/total_other_words)
    
    # store transition probabilities as a dictionary (i.e. a lookup table)
    # note: given the spell-checking algorithm, this cannot be maintained
    # as an RDD as it is not possible to map within a map
    transition_prob = transition_prob_calc.collectAsMap()
    
    ############
    #
    # generate dictionary
    #
    ############

    # note: this approach is slightly different from the original SymSpell
    # algorithm, but is more appropriate for a SPARK implementation
    
    # split into individual words (all)
    # RDD format: 'word1', 'word2', 'word3', ...
    # cache because this RDD is used in multiple operations 
    all_words = make_all_lower.map(lambda line: regex.sub(' ', line)) \
            .flatMap(lambda line: line.split()).cache()

    # use accumulator to count the number of words processed
    accum_words_processed = sc.accumulator(0)
    all_words.foreach(lambda x: accum_words_processed.add(1))

    # add a count to each word
    # RDD format: ('word1', 1), ('word2', 1), ('word3', 1), ...
    count_once = all_words.map(lambda word: (word, 1))

    # reduce into counts of unique words - this is the core corpus dictionary
    # (i.e. only words appearing in the file, without 'deletes'))
    # RDD format: ('word1', frequency), ('word2', frequency), ...
    # cache because this RDD is used in multiple operations 
    # note: imposing partitioning at this step yields a small 
    # improvement in runtime (~1 sec for big.txt) by equally
    # balancing elements among workers for subsequent operations
    unique_words_with_count = count_once.reduceByKey(lambda a, b: a + b, 
        numPartitions = num_partitions).cache()
    
    # use accumulator to count the number of unique words
    accum_unique_words = sc.accumulator(0)
    unique_words_with_count.foreach(lambda x: accum_unique_words.add(1))

    # generate list of "deletes" for each word in the corpus
    # RDD format: (word1, [deletes for word1]), (word2, [deletes for word2]), ...
    generate_deletes = unique_words_with_count.map(lambda (parent, count): 
        (parent, get_deletes_list(parent, max_edit_distance)))
    
    # split into all key-value pairs
    # RDD format: (word1, delete1), (word1, delete2), ...
    expand_deletes = generate_deletes.flatMapValues(lambda x: x)
    
    # swap word order and add a zero count (because "deletes" were not
    # present in the dictionary)
    swap = expand_deletes.map(lambda (orig, delete): (delete, ([orig], 0)))
    
    # create a placeholder for each real word
    # RDD format: ('word1', ([], frequency)), ('word2', ([], frequency)), ...
    corpus = unique_words_with_count.mapValues(lambda count: ([], count))

    # combine main dictionary and "deletes" (and eliminate duplicates)
    # RDD format: ('word1', ([deletes for word1], frequency)), 
    #             ('word2', ([deletes for word2], frequency)), ...
    combine = swap.union(corpus)
    
    # store dictionary items and deletes as a dictionary (i.e. a lookup table)
    # note: given the spell-checking algorithm, this cannot be maintained
    # as an RDD as it is not possible to map within a map
    # note: use reduceByKeyLocally to avoid an extra shuffle from reduceByKey
    dictionary = combine.reduceByKeyLocally(lambda a, b: (a[0]+b[0], a[1]+b[1])) 
    
    # output stats
    print 'Total words processed: %i' % accum_words_processed.value
    print 'Total unique words in corpus: %i' % accum_unique_words.value 
    print 'Total items in dictionary (corpus words and deletions): %i' \
        % len(dictionary)
    print '  Edit distance for deletions: %i' % max_edit_distance
    print 'Total unique words at the start of a sentence: %i' \
        % len(start_prob)
    print 'Total unique word transitions: %i' % len(transition_prob)
    
    return dictionary, start_prob, default_start_prob, \
            transition_prob, default_transition_prob

### Document correction

In [ ]:
######################
#
# SPELL-CHECKING - VITERBI ALGORITHM
#
# The below functions are used to read in a text file, break it down
# into individual sentences, and then carry out context-based spell-
# checking on each sentence in turn. In cases where the 'suggested'
# word does not match the actual word in the text, both the original
# and the suggested sentences are printed/outputed to file.
#
# Probabilistic model:
#
# Each sentence is modeled as a hidden Markov model, where the
# hidden states are the words that the user intended to type, and
# the emissions are the words that were actually typed.
#
# For each word in a sentence, we can define:
#
# - emission probabilities: P(observed word|intended word)
#
# - prior probabilities (for first words in sentences only):
# P(being the first word in a sentence)
#
# - transition probabilities (for all subsequent words):
# P(intended word|previous intended word)
#
# Prior and transition probabilities were calculated in the pre-
# processing steps above, using the same corpus as the dictionary.
# 
# Emission probabilities are calculated on the fly using a Poisson
# distribution as follows:
# P(observed word|intended word) = PMF of Poisson(k, l), where
# k = edit distance between word typed and word intended, and l=0.01.
# Both the overall approach and the parameter of l=0.01 are based on
# the 2015 lecture notes from AM207 Stochastic Optimization.
# Various parameters for lambda between 0 and 1 were tested, which
# confirmed that 0.01 yields the most accurate word suggestions.
#
# All probabilities are stored in log-space to avoid underflow. Pre-
# defined minimum values (also defined at the pre-processing stage)
# are used for words that are not present in the dictionary and/or
# probability tables.
#
# Algorithm:
#
# The spell-checking itself is carried out using a modified version
# of the Viterbi algorithm, which yields the most likely sequence of
# hidden states, i.e. the most likely sequence of words that form a
# sentence. The main difference to the 'standard' Viterbi algorithm
# is that the state space (i.e. the list of possible corrections) is
# generated (and therefore varies) for each word. This is in contrast
# to the alternative of considering the state space of all possible
# words in the dictionary for every word that is checked, which would
# be intractable for larger dictionaries.
#
# Example:
#
# The algorithm is best illustrated by way of an example.
#
# Suppose that we are checking the sentence 'This is ax test.'
# The emissions for the entire sentence are 'This is ax test.' and
# the hidden states for the entire sentence are 'This is a test.'
#
# As a pre-processing step, we convert everything to lowercase,
# eliminate punctuation, and break the sentence up into a list of
# words: ['this', 'is', 'ax', 'text']
# This list is passed as a parameter to the viterbi function.
#
# The algorithm tackles each word in turn, starting with 'this'.
#
# We first use get_suggestions to obtain a list of all words that
# may have been intended instead of 'this', i.e. all possible hidden
# states (intended words) for the emission (word typed).
#
# get_suggestions returns the 10 most likely corrections:
# - 1 word with an edit distance of 0
#   ['this']
# - 3 words with an edit distance of 1
#   ['his', 'thus', 'thin']
# - 6 words with an edit distance of 2 
#   ['the', 'that', 'is', 'him', 'they', 'their']
# 
# These 10 words represent our state space, i.e. possible words that
# may have been intended, and are referred to below as the list of
# possible corrections. They each have an emission probability equal
# to the PMF of Poisson(edit distance, 0.01).
#
# For each word in the list of possible corrections, we calculate:
# P(word starting a sentence) * P(observed 'this'|intended word)
# This is a simple application of Bayes' rule: by normalizing the
# probabilities we obtain P(intended word|oberved 'this') for
# each of the 10 words.
#
# We store the word-probability pairs for future use, and move on to
# the next word. 
#
# After the first word, all subsequent words are treated as follows.
#
# The second word in our test sentence is 'is'. Once again, we use
# get_suggestions to obtain a list of all words that may have been
# intended. get_suggestions returns the 10 most likely suggestions:
# - 1 word with an edit distance of 0
#   ['is']
# - 9 words with an edit distance of 1
#   ['in', 'it', 'his', 'as', 'i', 's', 'if', 'its', 'us']
# These 10 words represent our state space for the second word.
#
# For each word in the current list of possible corrections, we loop
# through all the words in the previous list of possible corrections,
# and calculate:
#    probability(previous suggested word) 
#    * P(current suggested word|previous suggested word)
#    * P(typing 'is'|meaning to type current suggested word)
# We determine which previous word maximizes this calculation and
# store that 'path' and probability for each current suggested word.
#
# For example, suppose that we are considering the possibility that
# 'is' was indeed intended to be 'is'. We then calculate: 
#    probability(previous suggested word)
#    * P('is'|previous suggested word) * P('is'|'is')
# for all previous suggested words, and discover that the previous
# suggested word 'this' maximizes the above calculation. We therefore
# store 'this is' as the optimal path for the suggested correction
# 'is' and the above (normalized) probability associated with this
# path.
#
# If the sentence had been only 2 words long, then at this point we
# would return the path that maximizes the most probability for the
# most recent step (word).
#
# As it is not, we repeat the previous steps for 'ax' and 'test',
# and then return the path that is associated with the highest
# probability at the last step.
#
######################

def dameraulevenshtein(seq1, seq2):
    '''
    Calculate the Damerau-Levenshtein distance between sequences.

    codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1
    matrix. However, only the current and two previous rows are
    needed at once, so we only store those.

    Same code as word-level checking.
    '''
    
    oneago = None
    thisrow = range(1, len(seq2) + 1) + [0]
    
    for x in xrange(len(seq1)):
        
        twoago, oneago, thisrow = \
            oneago, thisrow, [0] * len(seq2) + [x + 1]
        
        for y in xrange(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)

            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                and seq1[x-1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
                
    return thisrow[len(seq2) - 1]

def get_suggestions(string, dictionary, max_edit_distance, 
                    longest_word_length=20, min_count=100, max_sug=10):
    '''
    Return list of suggested corrections for potentially incorrectly
    spelled word.

    Code based on get_suggestions function from word-level checking,
    with the addition of the min_count and max_sug parameters.
    - min_count: minimum number of times a word must have appeared
    in the dictionary corpus to be considered a valid suggestion
    - max_sug: number of suggestions that are returned (ranked by
    frequency of appearance in dictionary corpus and edit distance
    from word being checked)

    These changes were imposed in order to ensure that the problem
    remains tractable when checking very large documents. In practice,
    the "correct" suggestion is almost always amongst the top ten.

    '''
    
    if (len(string) - longest_word_length) > max_edit_distance:
        # to ensure Viterbi can keep running -- use the word itself
        return [(string, 0)]
    
    suggest_dict = {}
    
    queue = [string]
    q_dictionary = {}  # items other than string that we've checked
    
    while len(queue)>0:
        q_item = queue[0]  # pop
        queue = queue[1:]
        
        # process queue item
        if (q_item in dictionary) and (q_item not in suggest_dict):
            if (dictionary[q_item][1]>0):
            # word is in dictionary, and is a word from the corpus,
            # and not already in suggestion list so add to suggestion
            # dictionary, indexed by the word with value (frequency
            # in corpus, edit distance)
            # note: q_items that are not the input string are shorter
            # than input string since only deletes are added (unless
            # manual dictionary corrections are added)
                assert len(string)>=len(q_item)
                suggest_dict[q_item] = \
                    (dictionary[q_item][1], len(string) - len(q_item))
            
            # the suggested corrections for q_item as stored in
            # dictionary (whether or not q_item itself is a valid
            # word or merely a delete) can be valid corrections
            for sc_item in dictionary[q_item][0]:
                if (sc_item not in suggest_dict):
                    
                    # compute edit distance
                    # suggested items should always be longer (unless
                    # manual corrections are added)
                    assert len(sc_item)>len(q_item)
                    # q_items that are not input should be shorter
                    # than original string 
                    # (unless manual corrections added)
                    assert len(q_item)<=len(string)
                    if len(q_item)==len(string):
                        assert q_item==string
                        item_dist = len(sc_item) - len(q_item)

                    # item in suggestions list should not be the same
                    # as the string itself
                    assert sc_item!=string           
                    # calculate edit distance using Damerau-
                    # Levenshtein distance
                    item_dist = dameraulevenshtein(sc_item, string)
                    
                    if item_dist<=max_edit_distance:
                        # should already be in dictionary if in
                        # suggestion list
                        assert sc_item in dictionary  
                        # trim list to contain state space
                        if (dictionary[q_item][1]>0): 
                            suggest_dict[sc_item] = \
                                (dictionary[sc_item][1], item_dist)
        
        # now generate deletes (e.g. a substring of string or of a
        # delete) from the queue item as additional items to check
        # -- add to end of queue
        assert len(string)>=len(q_item)
        if (len(string)-len(q_item))<max_edit_distance \
            and len(q_item)>1:
            for c in range(len(q_item)): # character index        
                word_minus_c = q_item[:c] + q_item[c+1:]
                if word_minus_c not in q_dictionary:
                    queue.append(word_minus_c)
                    # arbitrary value to identify we checked this
                    q_dictionary[word_minus_c] = None

    # return list of suggestions: (correction, edit distance)
    
    # only include words that have appeared a minimum number of times
    # note: make sure that we do not lose the original word
    as_list = [i for i in suggest_dict.items() 
               if (i[1][0]>min_count or i[0]==string)]
    
    # only include the most likely suggestions (based on frequency
    # and edit distance from original word)
    trunc_as_list = sorted(as_list, 
            key = lambda (term, (freq, dist)): (dist, -freq))[:max_sug]
    
    if len(trunc_as_list)==0:
        # to ensure Viterbi can keep running
        # -- use the word itself if no corrections are found
        return [(string, 0)]
        
    else:
        # drop the word frequency - not needed beyond this point
        return [(i[0], i[1][1]) for i in trunc_as_list]

    '''
    Output format:
    get_suggestions('file', dictionary)
    [('file', 0), ('five', 1), ('fire', 1), ('fine', 1), ('will', 2),
    ('time', 2), ('face', 2), ('like', 2), ('life', 2), ('while', 2)]
    '''
    
def get_emission_prob(edit_dist, poisson_lambda=0.01):
    '''
    The emission probability, i.e. P(observed word|intended word)
    is approximated by a Poisson(k, l) distribution, where 
    k=edit distance between the observed word and the intended
    word and l=0.01.
    
    Both the overall approach and the parameter of l=0.01 are based on
    the 2015 lecture notes from AM207 Stochastic Optimization.
    Various parameters for lambda between 0 and 1 were tested, which
    confirmed that 0.01 yields the most accurate word suggestions.
    '''
    
    return math.log(poisson.pmf(edit_dist, poisson_lambda))

######################
# Multiple helper functions are used to avoid KeyErrors when
# attempting to access values that are not present in dictionaries,
# in which case the previously specified default value is returned.
######################

def get_start_prob(word, start_prob, default_start_prob):
    '''
    P(word being at the beginning of a sentence)
    '''
    try:
        return start_prob[word]
    except KeyError:
        return default_start_prob
    
def get_transition_prob(cur_word, prev_word, 
                        transition_prob, default_transition_prob):
    '''
    P(word|previous word)
    '''
    try:
        return transition_prob[prev_word][cur_word]
    except KeyError:
        return default_transition_prob 

def get_sentence_word_id(words):
    '''
    Helper function: numbers each word according to its position
    in the sentence.
    '''
    return [(i, w) for i, w in enumerate(words)]

def start_word_prob(words, tmp_sp, d_sp):
    '''
    Helper function: calculates the probability of all word 
    suggestions being at the beginning of the sentence, based on
    the pre-processed start probabilities and the emission model.
    i.e. start probability x emission probability
    '''
    orig_word, sug_words = words
    probs = [(w[0], math.exp(
                get_start_prob(w[0], tmp_sp, d_sp) 
                + get_emission_prob(w[1])
            )) 
             for w in sug_words]
    sum_probs = sum([p[1] for p in probs])
    probs = [([p[0]], math.log(p[1]/sum_probs)) for p in probs]
    return probs

def split_suggestions(sentence):
    '''
    Helper function: Splits into all the suggestions for a given
    word, while retaining the previous path for all elements.
    '''
    sent_id, (word, word_sug)  = sentence
    return [[sent_id, (word, w)] for w in word_sug]

def normalize(probs):
    '''
    Helper function: normalizes probability so they add to 1.
    Note: this is especially necessary given the small
    probabilities that apply to this problem.
    '''
    sum_probs = sum([p[1] for p in probs])
    return [(p[0], math.log(p[1]/sum_probs)) for p in probs]

def get_max_prev_path(words, tmp_tp, d_tp):
    '''
    Helper function: Calculates the previous path that maximizes
    the probability of the current word suggestion.
    '''

    # unpack values
    cur_word = words[0][0]
    cur_sug = words[0][1][0]
    cur_sug_ed = words[0][1][1]
    prev_sug = words[1]
    
    # belief + transition probability + emission probability
    (prob, word) = max((p[1]
                 + get_transition_prob(cur_sug, p[0][-1], tmp_tp, d_tp)
                 + get_emission_prob(cur_sug_ed), p[0])
                     for p in prev_sug)
    
    return word + [cur_sug], math.exp(prob)

def get_max_path(final_paths):
    '''
    Helper function: at the final step, identifies the full path
    (i.e. sentence correction) with the highest probability.
    '''
    return max((p[1], p[0]) for p in final_paths)[1]

def get_count_mismatches(sentences):
    '''
    Helper function: compares the original sentence with the sentence
    that has been suggested by the Viterbi algorithm, and calculates
    the number of words that do not match.
    '''
    orig_sentence, sug_sentence = sentences
    count_mismatches = len([(orig_sentence[i], sug_sentence[i]) 
            for i in range(len(orig_sentence))
            if orig_sentence[i]!=sug_sentence[i]])
    return count_mismatches, orig_sentence, sug_sentence

def correct_document_context_parallel_full(fname, dictionary,
                             start_prob, default_start_prob,
                             transition_prob, default_transition_prob,
                             max_edit_distance=3, num_partitions=6,
                             display_results=False):
    
    '''
    Load a text file and spell-check each sentence using the
    dictionary and probability tables that were created in the
    pre-processing stage.

    Suggested corrections are either printed to the screen or
    saved in a log file, depending on the settings.
    '''

    ############
    #
    # load file & initial processing
    #
    ############
    
    # http://stackoverflow.com/questions/22520932/python-remove-all-non-alphabet-chars-from-string
    regex = re.compile('[^a-z ]')
    
    # broadcast Python dictionaries to workers (from pre-processing)
    bc_dictionary = sc.broadcast(dictionary)
    bc_start_prob = sc.broadcast(start_prob)
    bc_transition_prob = sc.broadcast(transition_prob)
    
    # load file contents and convert into one long sequence of words
    # RDD format: 'line 1', 'line 2', 'line 3', ...
    make_all_lower = sc.textFile(fname) \
            .map(lambda line: line.lower()) \
            .filter(lambda x: x!='')
    
    # split into individual sentences and remove other punctuation
    # RDD format: [words of sentence1], [words of sentence2], ...
    # cache because this RDD is used in multiple operations 
    split_sentence = make_all_lower.flatMap(lambda 
        line: line.replace('?','.').replace('!','.').split('.')) \
             .map(lambda sentence: regex.sub(' ', sentence)) \
             .map(lambda sentence: sentence.split()) \
             .filter(lambda x: x!=[]).cache()
    
    # use accumulator to count the number of words checked
    accum_total_words = sc.accumulator(0)
    split_sentence.flatMap(lambda x: x) \
            .foreach(lambda x: accum_total_words.add(1))
    
    # assign a unique id to each sentence
    # RDD format: (0, [words of sentence1]), (1, [words of sentence2]), ...
    # partition and cache here after completing transformations - this
    # RDD is used in multiple operations and the sentence id will
    # remain the key from this point forward
    sentence_id = split_sentence.zipWithIndex().map(
        lambda (k, v): (v, k)).partitionBy(num_partitions).cache()
    
    # count the number of words in each sentence - this is used to
    # determine when each sentence is done processing
    # RDD format: (0, words in sentence1), (1, words in sentence2), ...
    # cache as this RDD is called at every iteration
    sentence_word_count = sentence_id.mapValues(lambda v: len(v)).cache()
    
    ############
    #
    # spell-checking
    #
    ############

    # number each word in a sentence, and split into individual words
    # RDD format: (sentence1 id, (word1 id, word1)), 
    #             (sentence1 id, (word2 id, word2), ...
    sentence_word_id = sentence_id.mapValues(lambda v: get_sentence_word_id(v)) \
            .flatMapValues(lambda x: x)
   
    # get suggestions for each word
    # RDD format: (sentence1 id, (word1 id, word1, [suggestions for word1])), 
    #             (sentence1 id, (word2 id, word2, [suggestions for word2]), ...
    # cache as this RDD is called at each iteration
    sentence_word_suggestions = sentence_word_id.mapValues(
        lambda v: (v[0], v[1], get_suggestions(v[1], bc_dictionary.value,
            max_edit_distance))).cache()

    # filter for all the first words in sentences
    # RDD format: (sentence id, (0, word, [suggestions for word])), 
    #             (sentence id, (0, word, [suggestions for word]), ...
    sentence_word_1 = sentence_word_suggestions.filter(lambda (k, v): v[0]==0) \
            .mapValues(lambda v: (v[1], v[2]))

    # calculate probability for each suggestion
    # RDD format: (sentence id, [([word], P(word)), ([word], P(word)), ...]), 
    #             (sentence id, [([word], P(word)), ([word], P(word)), ...]), ...
    sentence_path = sentence_word_1.mapValues(lambda v: 
            start_word_prob(v, bc_start_prob.value, default_start_prob))

    # start loop from second word (zero-indexed)
    word_num = 1

    # extract any sentences that have been fully processed
    # RDD format: (sentence id, [([path], P(path)), ([path], P(path)), ...]), 
    #             (sentence id, [([path], P(path)), ([path], P(path)), ...]), ...
    completed = sentence_word_count.filter(lambda (k, v): v==word_num) \
            .join(sentence_path).mapValues(lambda v: v[1]) \
            .partitionBy(num_partitions).cache()
    
    # filter for the next words in sentences
    # RDD format: (sentence id, (word, [suggestions for word])), 
    #             (sentence id, (word, [suggestions for word]), ...
    sentence_word_next = sentence_word_suggestions.filter(lambda 
        (k,v): v[0]==word_num) \
            .mapValues(lambda v: (v[1], v[2])).cache()
    
    # check whether there are any words left to process
    while not sentence_word_next.isEmpty():

        # split by suggestions, while retaining previous path
        # RDD format: (sentence id, (word, (suggested word, edit distance)), 
        #                    [previous path]), ...
        # use preservesPartitioning to signal that the sentence id
        # continues to be the key
        sentence_word_next_split = sentence_word_next.flatMap(lambda x: 
            split_suggestions(x), preservesPartitioning=True)
        
        # join each suggestion with the previous path
        # RDD format:
        # (sentence id, ((current word, 
        #          (current word suggestion, edit distance)), 
        #               [(previous path-probability pairs)])), ...
        sentence_word_next_path = sentence_word_next_split.join(sentence_path)

        # identify previous path that maximizes the probability 
        # of each suggested word correction
        # RDD format: (sentence id, ([path], path probability)),
        #             (sentence id, ([path], path probability)), ...
        sentence_word_next_path_prob = sentence_word_next_path \
            .mapValues(lambda v: get_max_prev_path(v, 
                bc_transition_prob.value, default_transition_prob))

        # group all the new paths for each sentence and normalize
        # for numerical stability
        # RDD format: (sentence id, [([path], P(path)), ([path], P(path)), ...]), 
        #             (sentence id, [([path], P(path)), ([path], P(path)), ...]), ...
        # cache as this is used in multiple operations
        sentence_path = sentence_word_next_path_prob.groupByKey() \
                .mapValues(lambda v: normalize(v)).cache()

        # move on to next word
        word_num += 1
        
        # extract any sentences that have been fully processed
        # RDD format: (sentence id, [([path], P(path)), ([path], P(path)), ...]), 
        #             (sentence id, [([path], P(path)), ([path], P(path)), ...]), ...
        # cache as this is carried over to the next iteration
        # note: we confirmed that the RDDs being joined/unioned are
        #  co-partitioned during the development phase
        completed = completed \
            .union(sentence_word_count.filter(lambda (k, v): v==word_num) \
            .join(sentence_path) \
            .mapValues(lambda v: v[1])).cache()
       
        # filter for the next words in sentences
        # RDD format: (sentence id, (word, [suggestions for word])), 
        #             (sentence id, (word, [suggestions for word]), ...
        # cache as this is carried over to the next iteration
        sentence_word_next = sentence_word_suggestions.filter(
            lambda (k, v): v[0]==word_num) \
                .mapValues(lambda v: (v[1], v[2])).cache()

    # this is necessary for stability - otherwise too many threads
    # are spawned if we collect everything directly below
    completed.cache()

    # get most likely path (sentence)
    # RDD format: (sentence id, [suggested sentence]),
    #             (sentence id, [suggested sentence]), ...
    sentence_suggestion = completed.mapValues(lambda v: get_max_path(v))

    # checks that RDDs are co-partitioned
    # assert sentence_id.partitioner == sentence_suggestion.partitioner

    # join with original path (sentence)
    # RDD format: (sentence id, ([original sentence], [suggested sentence])),
    #             (sentence id, ([original sentence], [suggested sentence])), ...
    sentence_max_prob = sentence_id.join(sentence_suggestion)
        
    ############
    #
    # output results
    #
    ############
    
    # count the number of errors per sentence and drop any sentences
    # without errors
    # RDD format: (sentence id, (# errors, [original sentence], [suggested sentence])),
    #             (sentence id, (# errors, [original sentence], [suggested sentence])), ...
    sentence_errors = sentence_max_prob.mapValues(
        lambda v: get_count_mismatches(v)) \
            .filter(lambda (k, v): v[0]>0)

    # collect all sentences with identified errors (as list)
    sentence_errors_list = sentence_errors.collect()

    # count the number of potentially misspelled words
    num_errors = sum([s[1][0] for s in sentence_errors_list])
    
    # print suggested corrections
    if display_results:
        for sentence in sentence_errors_list:
            print 'Sentence %i: %s --> %s' % (sentence[0],
                ' '.join(sentence[1][1]), ' '.join(sentence[1][2]))
            print '-----'
    
    # output suggested corrections to file
    else:
        f = open('spell-log.txt', 'w')
        for sentence in sentence_errors_list:
            f.write('Sentence %i: %s --> %s\n' % (sentence[0], 
                ' '.join(sentence[1][1]), ' '.join(sentence[1][2])))
        f.close()
    
    print '-----'
    print 'Total words checked: %i' % accum_total_words.value
    print 'Total potential errors found: %i' % num_errors

### Sample performance

In [ ]:
dictionary_file = 'testdata/big.txt'
check_file = 'testdata/yelp100reviews.txt'

In [ ]:
print 'Pre-processing with %s...' % dictionary_file

start_time = time.time()

dictionary, start_prob, default_start_prob, transition_prob, default_transition_prob = \
    parallel_create_dictionary(dictionary_file)

run_time = time.time() - start_time

print '-----'
print '%.2f seconds to run' % run_time
print '-----'

```
Pre-processing with testdata/big.txt...
Total words processed: 1105285
Total unique words in corpus: 29157
Total items in dictionary (corpus words and deletions): 2151998
  Edit distance for deletions: 3
Total unique words at the start of a sentence: 15356
Total unique word transitions: 27086
-----
54.25 seconds to run
-----
```

In [ ]:
print 'Spell-checking %s...' % check_file

start_time = time.time()

correct_document_context_parallel_full(check_file, dictionary,
                        start_prob, default_start_prob, 
                        transition_prob, default_transition_prob)

run_time = time.time() - start_time

print '-----'
print '%.2f seconds to run' % run_time
print '-----'

```
Spell-checking testdata/yelp100reviews.txt...
-----
Total words checked: 12029
Total potential errors found: 1735
-----
367.10 seconds to run
-----
```
Sample output with suggested corrections <a href='sample/context-spell-log-yelp.txt'>here</a>.

### Notes on SPARK optimization

Unlike the previous implementation, the complexity of this approach meant that there was significant scope for optimization. By experimenting with various partitioning and caching strategies, we were able to materially increase the number of words checked per second. The motivation for the changes that were incorporated in the final version are included in-line with the code above.

One noteworthy point is the memory complexity that is introduced by iterating through each word position for each sentence in parallel (i.e. each RDD element in this iteration is a single word, as opposed to a sentence is the first implementation). When first testing with larger file sizes, we found that this resulted in the code spawning more threads than could be handled by our local configurations. To alleviate this problem, we introduced an additional caching step of the "completed" sentences when exiting the loop. This increased the runtime for smaller files sizes, but meant that we were able to run larger file sizes locally.

Despite all the improvements, the additional complexity of this implementation means that it remains slower overall for all file sizes compared to the naive parallelization approach.

As with the previous implementation, we found that a relatively small number of partitions (6) performed consistently well across file sizes.

<img src='img/contextSPARK3optimization.jpg' width=75% height=75%>

### Notes on local & AWS performance

The serial implementation consistently checks approximately 40 words per second. For smaller file sizes, this was clearly superior to the performance of the SPARK code up to a file size of 100 Yelp reviews (33 words per second in SPARK). However, the SPARK code began to outperform the serial code when moving to larger file sizes, and was able to reach 47 words per second when testing the file with 500 Yelp reviews. This file was the largest that could be run locally due to memory constraints.

Moving to AWS yielded further improvements, though this implementation did not scale as well as the previous one. For example, the smallest cluster tested (4 executors x 4 cores) was able to achieve 110 words per second on the same 500-review file - just over 2 times the local speed (compared to the 4x speed-up for the first implementation). This improved only marginally to 115 words per second when moving to the 1,000-review file. However, it is interesting to note that increasing the number of partitions to 64 from 16 resulted in further improvements to 130 words per second. Unlike the first implementation, which performed best on this small cluster configuration with 16 partitions, the higher number of RDD elements required for this implementation is better suited to a higher number of partitions, especially when processing larger file sizes.

Moving to a medium-size cluster (8 executors x 4 cores) resulted in a ~1.7x improvement to 215 words per second, while the large cluster (16 executors x 4 cores) yielded a ~2.2x improvement to 289 words per second (both with respect to the small cluster on the 1,000-review file). This further confirms that this method does not scale to larger cluster sizes as effectively as the first SPARK implementation.

Once again, we used m3.large instances for all of the above configurations, and upgraded the default AWS Python build to version 2.7 for consistency with the local development environment.

<img src='img/contextSPARK3performance.jpg' width=50% height=50%>